# PODMAN IN ACTION: Secure, rootless containers for Kubernetes, microservices, and more

by Daniel Walsh

- Publisher : Manning (February 7, 2023)
- Language : English
- Paperback : 312 pages
- ISBN-10 : `1633439682`
- ISBN-13 : `978-1633439689`

Example image:

```sh
podman run -d -p 8080:80 --name myapp docker.io/library/httpd
podman exec -i myapp bash -c 'cat > /usr/local/apache2/htdocs/index.html' << _EOF
<html>
 <head>
 </head>
 <body>              
  <h1>Rot Front!</h1>
 </body>
</html>
_EOF
```

# <b>1. Podman: A next-generation container engine</b>

## 1.1 About all these terms

Before you go further, I think it is important to define the terminology that will be used throughout this book. In the container world, terms like container orchestrator, container engine, and container runtime are often used interchangeably, which commonly leads to confusion. The following list is a summary of what each of these terms refers to in the context of this text:

### Container orchestrators

Software projects and products that orchestrate containers onto multiple different machines or nodes. These orchestrators communicate with **container engines** to run containers. The primary container orchestrator is **Kubernetes**, which was originally designed to talk to the Docker daemon container engine, but using Docker is becoming obsolete because Kubernetes primarily uses CRI-O or containerd as its container engine. **CRI-O** and **containerd** are purpose built for running orchestrated Kubernetes containers (CRI-O is covered in appendix A). Docker Swarm and Apache Mesos are other examples of container orchestrators.

#### CRI-O

_ChatGPT:_  
**CRI-O** is an open-source, lightweight container runtime for Kubernetes. It is designed to provide a stable, secure, and scalable environment for running containers in a Kubernetes cluster. CRI-O is optimized for use with Kubernetes and supports the **container runtime interface (CRI)** standard, allowing it to seamlessly integrate with the Kubernetes ecosystem. It is an alternative to other container runtimes like Docker and is specifically built to meet the requirements of Kubernetes container orchestration. CRI-O focuses on minimalism, performance, and security, making it a popular choice for running containers in production environments.

#### Container Runtime Interface

_ChatGPT:_  
**Container Runtime Interface (CRI)** is an interface between 
- container runtimes and 
- container management systems (e.g. Kubernetes). 

It allows container runtimes to be pluggable and allows different container runtimes to be used with the same container management system. This flexibility enables organizations to choose the best container runtime for their needs and easily switch between runtimes as necessary. CRI defines a set of APIs that container runtimes must implement in order to be compatible with container orchestration systems like Kubernetes.

#### Containerd

_ChatGPT:_  
**Containerd** is an open-source container runtime that provides a runtime environment for running containerized applications. It is designed to be lightweight, portable, and platform agnostic, making it suitable for use in various environments, including cloud, on-premises, and edge computing.

Containerd is used as the container runtime within Docker, as well as by other container orchestration platforms such as Kubernetes. It provides core container runtime functionalities, such as image distribution, container execution, and storage management. Containerd is designed with a modular architecture, allowing it to be integrated into different container management systems and to support various container formats.

In summary, Containerd is a fundamental component in the container ecosystem, providing essential container runtime functionalities while enabling interoperability and versatility across different container platforms.

#### Containerd vs CRI-O

- Architecture:
    - **Containerd:** Containerd is a core container runtime that provides a runtime environment for running containerized applications. It is designed to be lightweight and platform-agnostic, focusing on managing the container lifecycle, including 
        - image distribution, 
        - container execution, and 
        - storage management.  
        Containerd can be used as a building block for higher-level container orchestration systems.
    - **CRI-O:** CRI-O, on the other hand, is specifically designed for Kubernetes container runtimes. It implements the Container Runtime Interface (CRI) and focuses on creating an optimized runtime for Kubernetes pods and containers. CRI-O is specifically tailored for integrating with Kubernetes and adheres to the Kubernetes container runtime standards.<br><br>
- Integration with Kubernetes:
    - **Containerd:** Containerd can be integrated with Kubernetes as a container runtime, providing the core functionality required for running containers within a Kubernetes cluster.
    - **CRI-O:** CRI-O is designed as a lightweight and optimized runtime for Kubernetes. It directly implements the CRI, allowing Kubernetes to use CRI-O as a container runtime without the need for additional layers.<br><br>
- Feature Set:
    - **Containerd:** As a fundamental container runtime, Containerd provides core container lifecycle management functionalities, including image distribution, container execution, and storage management. It focuses on being modular and platform-agnostic, suitable for use in various container management systems.
    - **CRI-O:** CRI-O is specifically tailored to address the requirements of Kubernetes. It is designed to efficiently handle Kubernetes pod and container management, adhering closely to the Kubernetes container runtime standards while providing essential container runtime features.

In summary, Containerd is a general-purpose, lightweight container runtime suitable for various container management systems, while CRI-O is specifically optimized for Kubernetes, providing a streamlined and Kubernetes-focused runtime environment. Both container runtimes have unique strengths based on their intended use cases and integrations.

### Container engines

Primarily used for configuring containerized applications to run _on a single local node_. They can be launched directly by users, administrators, and developers. They can also be launched out of systemd unit files at boot as well as launched by **container orchestrators** like Kubernetes. As previously mentioned, `CRI-O` and `containerd` are container engines (_runtimes_ - VR) used by Kubernetes to manage containers locally. They really are not intended to be used directly by users. `Docker` and `Podman` are the primary **container engines** used by users to develop, manage, and run containerized applications on a single machine. Podman is seldom used to launch containers for Kubernetes; therefore, Kubernetes is not generally covered in this book. `Buildah` is another container engine, although it is only used for building container images.

### Open Container Initiative (OCI) container runtimes

Configure different parts of the Linux kernel and then, finally, launch the containerized application. The two most commonly used container runtimes are `runc` and `crun`. `Kata` and `gVisor` are other examples of container runtimes. See appendix B to understand the differences between the OCI container runtimes.

_ChatGPT:_  
`runc` is not a full-fledged container runtime, but a lightweight command-line tool for spawning and running containers. It is a low-level tool used by higher-level container runtimes (like CRI-O and containerd) to create and run containers based on OCI (Open Container Initiative) specifications.

### Podman

**Podman** is short for _Pod Manager_. A **pod**, a concept popularized by the Kubernetes project, is one or more containers sharing the same namespaces and cgroups (resource constraints). Pods are covered in greater depth in **chapter 4**. Podman runs individual containers as well as pods. The Podman logo is a group of Selkies, the Irish concept of a mermaid. Groups of Selkies are called pods.

![](../data/images/podman-logo-2009470014.png)

![](../data/images/fgfff54fdgdg.jpg)

![](../data/images/048c64b76bdb2b40157dc81f6d246e96.png)

## 1.2 A brief overview of containers

### Containers

**Containers** are groups of _processes_ running on a Linux system that are isolated from each other. 

Containers make sure one group of processes does not interfere with other processes on the system. Rogue processes can’t dominate system resources, which might prevent other processes from performing their task. Hostile containers are also prevented from attacking other containers, stealing data, or causing denial of service attacks. 

> A final goal of containers is allowing applications to be installed with their own versions of shared libraries that do not conflict with applications requiring different versions of the same libraries. 

Instead they allow applications to live in a virtualized environment, giving the impression that they own the entire system.

Containers are isolated via the following:

#### Resource constraints (cgroups)

The [cgroup man page](https://man7.org/linux/man-pages/man7/cgroups.7.html) defines cgroups as the following: 

_“Control groups, usually referred to as cgroups, are a Linux kernel feature which allow processes to be organized into hierarchical groups whose usage of various types of resources can then be limited and monitored.”_

Examples of resources controlled by cgroups include the following:

- The amount of **memory** a group of processes can use,
- The amount of **CPU** processes can use,
- The amount of **network** resources a process can use.

The basic idea of cgroups is 

> preventing one group of processes from dominating certain system resources in such a way that another group of processes can’t make progress on the system.

In [12]:
man cgroups

cgroups(7)             Miscellaneous Information Manual             cgroups(7)

NAME
       cgroups - Linux control groups

DESCRIPTION
       Control groups, usually referred to as cgroups, are a Linux kernel fea‐
       ture which allow processes to be  organized  into  hierarchical  groups
       whose usage of various types of resources can then be limited and moni‐
       tored.  The kernel's cgroup interface is  provided  through  a  pseudo-
       filesystem called cgroupfs.  Grouping is implemented in the core cgroup
       kernel code, while resource tracking and limits are  implemented  in  a
       set of per-resource-type subsystems (memory, CPU, and so on).

   Terminology
       A cgroup is a collection of processes that are bound to a set of limits
       or parameters defined via the cgroup filesystem.

       A subsystem is a kernel component that modifies  the  behavior  of  the
       processes  in a cgroup.  Various subsystems have been implemented, mak‐
       ing 

#### Security constraints

Containers are isolated from each other using many security tools available in the kernel. The goal is 

> blocking privilege escalation and preventing a rogue group of processes from committing hostile acts against the system, including the following examples:

- Dropped Linux capabilities limit the power of root.
- SELinux controls access to the filesystem.
- There is read-only access to kernel filesystems.
- Seccomp limits the system calls available in the kernel.
- A user namespace to map one group of UIDs in the host to another allows access to limited root environments.

### Namespaces - Virtualization technologies

The Linux kernel employs a concept called **namespaces**, which creates virtualized environments, where one set of processes sees one set of resources, while another set of processes sees a different set of resources. These **virtualized environments** eliminate processes’ views into the rest of the system, giving them the feel of a virtual machine (VM) without the overhead. Examples of namespaces include the following:

- **Network namespace** — Eliminates the access to the host network but gives access to virtual network devices,
- **Mount namespace** — Eliminates the view of all the filesystem, except the containers filesystem,
- **PID namespace** — Eliminates the view of other processes on the system; container processes only see the processes within the container.

These container technologies have existed in the Linux kernel for many years. Security tools for isolating processes started in Unix back in the 1970s, and SELinux started in 2001. Namespaces were introduced around 2004, and cgroups were introduced around 2006.

### Container image format

A **container image** consists of three components:

- A **directory tree** containing all the software required to run your application,
- A JSON file that describes the contents of the **rootfs**,
- Another JSON file called a **manifest list** that links multiple images together to support different architectures.

The directory tree is called a **rootfs** (root filesystem). The software is laid out like it was the root (`/`) of a Linux system.

The executable to be run within the `rootfs`, the working directory, the environment variables to be used, the maintainer of the executable, and other labels to help identify the content of the image are defined in the first JSON file. You can see this JSON file using the `podman inspect` command:

```sh
$ podman inspect docker://registry.access.redhat.com/ubi8
```
```json
{
...
  "created": "2022-01-27T16:00:30.397689Z",      ❶
  "architecture": "amd64",                       ❷
  "os": "linux",                                 ❸
  "config": {
         "Env": [                                ❹
            "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin",
            "container=oci"
         ],
         "Cmd": [                                ❺
                   "/bin/bash"
         ],
         "Labels": {                             ❻
                     "architecture": "x86_64",
                     "build-date": "2022-01-27T15:59:52.415605",
       ...
}
```

1. Date the image was created
1. Architecture for this image
1. Operating system for this image
1. Environment variables that the developer of the image wants to be set within the container
1. Default command to be executed when the container starts
1. Labels to help describe the contents of the image. These fields can be free-form and do not affect the way images are run but can be used to search for and describe the image.

The second JSON file, the **manifest list**, allows users on an arm64 machine to pull an image with the same name as they would if they were on an arm64 machine. Podman pulls the image based on the default architecture of the machine, using this manifest list:

```json
{
    "manifests": [
      {
              "digest": "sha256:cbc1e8cea
➥ 8c78cfa1490c4f01b2be59d43ddbb
➥ ad6987d938def1960f64bcd02c",                                                    ❶
              "mediaType": "application/vnd.docker.distribution.manifest.v2+json",❷
              "platform": {
              "architecture": "amd64",                                            ❸
              "os": "linux"                                                       ❹
              },
              "size": 737
      },
      {
              "digest":                                                           ❺
➥ "sha256:f52d79a9d0a3c23e6ac4c3c8f2ed8d6337ea47f4e2dfd46201756160ca193308",
              "mediaType": "application/vnd.docker.distribution.manifest.v2+json",
              "platform": {
              "architecture": "arm64", 
              "os": "linux"
              },
              "size": 737
      },
...
}
```

1. Digest (hash sum) of the exact image pulled when the architecture and OS match
1. mediaType describes the type of the image, OCI, Docker, and so on.
1. The architecture of this image digest: amd64
1. The OS of this image digest: Linux
1. This stanza (строка файла конфигурации) points to a different image for a different architecture: arm64.

### <b>Skopeo</b>

**Skopeo** (Greek for "remote viewing") is a tool that uses the same underlying libraries as Podman and is available at [github.com/containers/skopeo](github.com/containers/skopeo) (see appendix A). Skopeo provides lower-level output examining the structures of a container image. 

Skopeo works with Podman and Buildah to manage OCI containers. Put simply, 
- **Podman** runs containers, 
- **Buildah** builds containers, and 
- **Skopeo** transports containers –– among other things. 

Think of these tools as a swiss army knife for your container environment. Skopeo is a deft and versatile blade at your disposal. ([source](https://www.redhat.com/en/topics/containers/what-is-skopeo))

In the following example, use the `skopeo` command with the `--raw` option to examine the [registry.access.redhat.com/ubi8](registry.access.redhat.com/ubi8) image manifest specification without the need to pull the image:

In [11]:
# sudo apt update && sudo apt install skopeo -y

skopeo inspect --raw docker://registry.access.redhat.com/ubi8 | jq

{
  "manifests": [
    {
      "digest": "sha256:4b5ad3e7af158ded8070d4e0f1d75a3f197d065efeadddf33455f486f30e7832",
      "mediaType": "application/vnd.docker.distribution.manifest.v2+json",
      "platform": {
        "architecture": "amd64",
        "os": "linux"
      },
      "size": 429
    },
    {
      "digest": "sha256:5c33b45ca04495caaa41eaae45e471748480d44fd39aa09350db45bbdeb40f55",
      "mediaType": "application/vnd.docker.distribution.manifest.v2+json",
      "platform": {
        "architecture": "arm64",
        "os": "linux"
      },
      "size": 429
    },
    {
      "digest": "sha256:47df466911a0351a1e9744fb065bab435f7f5aec0050ad0e764fbed45b4668fd",
      "mediaType": "application/vnd.docker.distribution.manifest.v2+json",
      "platform": {
        "architecture": "ppc64le",
        "os": "linux"
      },
      "size": 429
    },
    {
      "digest": "sha256:90a0be2052eaf5adcaff2f9ccbe2e0b4eeb4421454c7a1eb19d3c9e3d7cfc0b8",
      "mediaType": "application/vnd.doc

#### Postgres example (VR)

In [9]:
podman search postgres

NAME                                                 DESCRIPTION
docker.io/library/postgres                           The PostgreSQL object-relational database sy...
docker.io/bitnami/postgresql                         Bitnami PostgreSQL Docker Image
docker.io/cimg/postgres                              
docker.io/bitnami/postgres-exporter                  
docker.io/bitnami/postgresql-repmgr                  
docker.io/ubuntu/postgres                            PostgreSQL is an open source object-relation...
docker.io/rapidfort/postgresql                       RapidFort optimized, hardened image for Post...
docker.io/rapidfort/postgresql-official              RapidFort optimized, hardened image for Post...
docker.io/bitnamicharts/postgresql                   
docker.io/rapidfort/postgresql12-ib                  RapidFort optimized, hardened image for Post...
docker.io/cockroachdb/postgres-test                  An environment to run the CockroachDB accept...
docker.io/elestio/postgres  

In [9]:
podman images

REPOSITORY  TAG         IMAGE ID    CREATED     SIZE


Get the manifest file:

In [10]:
skopeo inspect --raw docker://docker.io/library/postgres | jq

{
  "manifests": [
    {
      "annotations": {
        "org.opencontainers.image.revision": "d416768b1a7f03919b9cf0fef6adc9dcad937888",
        "org.opencontainers.image.source": "https://github.com/docker-library/postgres.git#d416768b1a7f03919b9cf0fef6adc9dcad937888:16/bookworm",
        "org.opencontainers.image.url": "https://hub.docker.com/_/postgres",
        "org.opencontainers.image.version": "16.1"
      },
      "digest": "sha256:60c91e0203ae5ccca0a251953742752cd16a129db181bc15559cca71420b188c",
      "mediaType": "application/vnd.oci.image.manifest.v1+json",
      "platform": {
        "architecture": "amd64",
        "os": "linux"
      },
      "size": 3324
    },
    {
      "annotations": {
        "vnd.docker.reference.digest": "sha256:60c91e0203ae5ccca0a251953742752cd16a129db181bc15559cca71420b188c",
        "vnd.docker.reference.type": "attestation-manifest"
      },
      "digest": "sha256:d75fb3e8a4753c68a7ab34ebe8fef6b7a9c1daa76f202a9fc6fef45c9b844b82",
      "medi

Get the overall information of the repo:

In [6]:
skopeo inspect docker://docker.io/library/postgres | jq

{
  "Name": "docker.io/library/postgres",
  "Digest": "sha256:49c276fa02e3d61bd9b8db81dfb4784fe814f50f778dce5980a03817438293e3",
  "RepoTags": [
    "10",
    "10-alpine",
    "10-alpine3.13",
    "10-alpine3.14",
    "10-alpine3.15",
    "10-alpine3.16",
    "10-beta1",
    "10-beta1-alpine",
    "10-beta2",
    "10-beta2-alpine",
    "10-beta3",
    "10-beta3-alpine",
    "10-beta4",
    "10-beta4-alpine",
    "10-bullseye",
    "10-buster",
    "10-rc1",
    "10-rc1-alpine",
    "10-stretch",
    "10.0",
    "10.0-alpine",
    "10.1",
    "10.1-alpine",
    "10.10",
    "10.10-alpine",
    "10.11",
    "10.11-alpine",
    "10.12",
    "10.12-alpine",
    "10.13",
    "10.13-alpine",
    "10.14",
    "10.14-alpine",
    "10.15",
    "10.15-alpine",
    "10.16",
    "10.16-alpine",
    "10.17",
    "10.17-alpine",
    "10.17-alpine3.13",
    "10.17-alpine3.14",
    "10.17-buster",
    "10.17-stretch",
    "10.18",
    "10.18-alpine",
    "10.18-alpine3.14",
    "10.18-bullseye",
    "

Skip annoying tags:

In [5]:
skopeo inspect --no-tags docker://docker.io/library/postgres | jq

{
  "Name": "docker.io/library/postgres",
  "Digest": "sha256:49c276fa02e3d61bd9b8db81dfb4784fe814f50f778dce5980a03817438293e3",
  "RepoTags": [],
  "Created": "2024-01-04T21:52:40Z",
  "DockerVersion": "",
  "Labels": null,
  "Architecture": "amd64",
  "Os": "linux",
  "Layers": [
    "sha256:2f44b7a888fa005d07c031d3cfad2a1c0344207def2ab9dbb97712425ff812c1",
    "sha256:6d49150dabe2486c7b35dbe0ea864b690e079fa1b48ef77d1e4533d96b4051e7",
    "sha256:18d6a86d0fbff788e398befc7b6139bed448853e3a113ea6b2c6c97783f4d8a1",
    "sha256:4c9385c30bce0478ebb3d7b8cd3ae66e9b211e980669bd0d67f3bb38b8bfaa40",
    "sha256:550091272acc768f00ff9a7ae977f9c558f5d3364cd5f87fabcc452d6f677b0e",
    "sha256:2720859ac49e9505ac9b95a3937053a3e0c2bb623b4bb6f49468c20ef7db8bc4",
    "sha256:b8091cf535458ca73ced6bb49c25ec769ae88608216e721e6bd1767d36138e3f",
    "sha256:f3ca5fbd89cdb8212ec43875a45b14a9f1cfda38272896347251e76e0cc383fe",
    "sha256:22fbbce47a56e5ceed3d8ec4a588a1adaed23a83c230645206ec26d961f876cd",
    "s

Get only layers:

In [18]:
skopeo inspect docker://docker.io/library/postgres | jq .Layers

[
  "sha256:2f44b7a888fa005d07c031d3cfad2a1c0344207def2ab9dbb97712425ff812c1",
  "sha256:6d49150dabe2486c7b35dbe0ea864b690e079fa1b48ef77d1e4533d96b4051e7",
  "sha256:18d6a86d0fbff788e398befc7b6139bed448853e3a113ea6b2c6c97783f4d8a1",
  "sha256:4c9385c30bce0478ebb3d7b8cd3ae66e9b211e980669bd0d67f3bb38b8bfaa40",
  "sha256:550091272acc768f00ff9a7ae977f9c558f5d3364cd5f87fabcc452d6f677b0e",
  "sha256:2720859ac49e9505ac9b95a3937053a3e0c2bb623b4bb6f49468c20ef7db8bc4",
  "sha256:b8091cf535458ca73ced6bb49c25ec769ae88608216e721e6bd1767d36138e3f",
  "sha256:f3ca5fbd89cdb8212ec43875a45b14a9f1cfda38272896347251e76e0cc383fe",
  "sha256:22fbbce47a56e5ceed3d8ec4a588a1adaed23a83c230645206ec26d961f876cd",
  "sha256:b3b5e3b65b9594578082fe4f45b479c2de5782344f17d74b7e34a3184000bf28",
  "sha256:917e5b76e085bfd128b12de93138dc43366eefc6efe52bedea5d351a0612b50f",
  "sha256:7f21ce9572c6e1750cc335372d0d1c6a893da69e3064e7efc961237c5ba63538",
  "sha256:4ea3941c8572e8e5721645ca86d9b9ddc868a5276c899e036d6da84fba9890ec

Count the number of tags:

In [8]:
skopeo list-tags docker://docker.io/library/postgres | wc -l

828


#### Mans of Skopeo

In [13]:
man skopeo

SKOPEO(1)(Skopeo)                                            SKOPEO(1)(Skopeo)

Jhon Honce August 2016

NAME
       skopeo  --  Command line utility used to interact with local and remote
       container images and container image registries

SYNOPSIS
       skopeo [global options] command [command options]

DESCRIPTION
       skopeo is a command line utility providing various operations with con‐
       tainer images and container image registries.

       skopeo  can  copy  container  images  between  various containers image
       stores, converting them as necessary.  For example you can  use  skopeo
       to copy container images from one container registry to another.

       skopeo  can convert a Docker schema 2 or schema 1 container image to an
       OCI image.

       skopeo can inspect a repository on a container registry  without  need‐
       lessly  pulling  the  image.  Pulling an image from a repository, espe‐
       cially a remote repository, is an expensive networ

In [4]:
man skopeo-inspect

skopeo-inspect(1)()                                        skopeo-inspect(1)()

NAME
       skopeo-inspect  -  Return  low-level  information about image-name in a
       registry.

SYNOPSIS
       skopeo inspect [options] image-name

DESCRIPTION
       Return low-level information  about  image-name  in  a  registry.   See
       skopeo(1) for the format of image-name.

       The  default  output  includes  data  from  various sources: user input
       (Name), the remote repository, if any (RepoTags), the  top-level  mani‐
       fest  (Digest),  and  a  per-architecture/OS image matching the current
       run-time environment (most other values).  To see values for a  differ‐
       ent  architecture/OS,  use  the --override-os / --override-arch options
       documented in skopeo(1).

OPTIONS
       --authfile path

       Path of the authentication file. Default is ${XDG_RUNTIME_DIR}/contain‐
       ers/auth.json,  which  is set using skopeo login.  If the authorization
       s

In [10]:
man skopeo copy

skopeo-copy(1)()                                              skopeo-copy(1)()

NAME
       skopeo-copy  -  Copy an image (manifest, filesystem layers, signatures)
       from one location to another.

SYNOPSIS
       skopeo copy [options] source-image destination-image

DESCRIPTION
       Copy an image (manifest, filesystem layers, signatures) from one  loca‐
       tion to another.

       Uses  the  system's trust policy to validate images, rejects images not
       trusted by the policy.

       source-image use the "image name" format described above

       destination-image use the "image name" format described above

       source-image and destination-image are interpreted completely  indepen‐
       dently;  e.g.  the  destination name does not automatically inherit any
       parts of the source name.

OPTIONS
       --additional-tag=strings

       Additional tags (supports docker-archive).

       --all, -a

       If source-image refers to a list of images, instead of cop

### Container registries

Images use the Linux `tar` utility to pack the `rootfs` and the JSON files together. These images are then stored on web servers called **container registries** (e.g., `docker.io`, `quay.io`, and `Artifactory`). **Container engines** like Podman can copy these images to a host and unpack them onto the filesystem. Then the engine merges 
- the image’s JSON file, 
- the engine’s built-in defaults, and 
- the user’s input 

to create a new container OCI runtime specification JSON file. The JSON file describes how to run the containerized application.

### Container runtime

In the last step, the container engine launches a small program called a **container runtime** (e.g., `runc`, `crun`, `kata`, or `givisord`). The container runtime reads the 
- container’s JSON and instruments, 
- kernel cgroups, 
- security constraints, and 
- namespaces 

before finally launching the primary process of the container.

### Container standards

The **OCI standards** body defined the standard formats for storing and defining container images. They also defined the standard for container engines running containers. The OCI created 
- the **OCI Image Format**, which standardizes the format of the container images and the images’ JSON file; 
- the **OCI Runtime Specification**, which standardized the container’s JSON file to be used by OCI runtimes. 

The OCI standards allow other container engines, like Podman, to follow the standards and be able to work with all the images stored at container registries and to run them in the exact same way as all other container engines, including Docker (see figure 1.7).

### Pictures

<center>
    <img src="../data/images/Screenshot_20240114_010033.png" alt="Figure 1.4 Physical machine running three applications in three VMs" style="width: 450px">
    <p style="text-align: center"><i>Figure 1.4 Physical machine running three applications in three VMs</i></p>

<center>
    <img src="../data/images/Screenshot_20240114_010239.png" alt="Figure 1.5 Physical machine running three applications in three containerized applications" style="width: 450px">
    <p style="text-align: center"><i>Figure 1.5 Physical machine running three applications in three containerized applications</i></p>

![](../data/images/user-space-vs-kernel-space-virtualization-vs-containerization11.png)

[Image source](https://www.redhat.com/en/blog/architecting-containers-part-2-why-user-space-matters)

<center>
    <img src="../data/images/Screenshot_20240114_010402.png" alt="Figure 1.6 Traditional LAMP stack (Linux, Apache, MariaDB, and PHP/PERL application) running on a server" style="width: 450px">
    <p style="text-align: center"><i>Figure 1.6 Traditional LAMP stack (Linux, Apache, MariaDB, and PHP/PERL application) running on a server</i></p>

<center>
    <img src="../data/images/Screenshot_20240114_005714.png" alt="Figure 1.7 LAMP stack packaged individually into microservice containers. As containers communicate via the network, they can be easily moved to other VMs, making reuse much easier." style="width: 450px">
    <p style="text-align: center"><i>Figure 1.7 LAMP stack packaged individually into microservice containers. As containers communicate via the network, they can be easily moved to other VMs, making reuse much easier.</i></p>

## 1.3 `Fork`/`exec` model

Docker is built as a REST API server. Fundamentally Docker is a client-server architecture including multiple daemons. 
- When a user executes the `Docker client`, they execute a command-line tool that connects to the Docker daemon.
- The `Docker daemon` then pulls images to its storage and 
- then connects to the `containerd daemon`, which 
- finally executes an `OCI runtime` that creates 
- the `container`. 

The Docker daemon, then, is a communication platform that communicates reads and writes of `stdin`, `stdout`, and `stderr` from the initial process (`PID1`) created in the container. The daemon relays all of the output back to the Docker client. Users imagine the container’s processes are just children of the current session, but there is a lot of communication going on behind the scenes. Figure 1.8 shows the Docker client-server architecture (also see [Container](../containers_nomenclature.ipynb#Container) from "Containers Nomenclature").

The bottom line is 
- the Docker client communicates with the `Docker daemon`, which then 
- communicates with the `containerd daemon`, which finally
- launches an OCI runtime like `runc` to 
- launch PID1 of the `container`. 

There is a lot of complexity involved in running containers in this way. Over the years, failures in any of the Daemons have led to all containers shutting down, and it is often difficult to diagnose what happened. 

The core Podman engineering team comes from an operating system background grounded in the Unix philosophy.

Unix and C were designed with the **fork/exec model** of computing. Basically, when you execute a new program, a parent program like the Bash shell _forks_ a new process and then _executes_ the new program as a child of the old program. 

The Podman engineering team thought they could make containers simpler by building a tool that 
- pulls container images from a container registry, 
- configures container storage, and then 
- launches an `OCI runtime`, which 
- starts the container as a child of your container engine.

In the Unix operating system, processes can share content via the **filesystem** and **inter-process communication (IPC)** mechanisms. These features of the operating system enable multiple container engines to share storage _without requiring a daemon_ to be running to control access and share content. The engines do not need to communicate together aside from using locking mechanisms provided by the operating system’s filesystems. Future chapters examine the advantages and disadvantages of this mechanism. Figure 1.9 shows the Podman architecture and communication flow.

<center>
    <div style="display: flex; flex-direction: row">
        <div>
            <img src="../data/images/Screenshot_20240114_015956.png" alt="Figure 1.8 Docker client-server architecture. The container is a direct descendant of containerd, not the Docker client. The kernel sees no relationship between the client program and the container." style="width: 600px; padding: 5px;">
            <p style="text-align: center; padding-top: 0px"><i>Figure 1.8 Docker client-server architecture. The container is a direct descendant of containerd, not the Docker client. The kernel sees no relationship between the client program and the container.<i></p>
        </div>
        <div>
            <img src="../data/images/Screenshot_20240114_020023.png" alt="Figure 1.9 Podman fork/exec architecture. The user launches Podman, which executes the OCI runtime, which then launches the container. The container is a direct descendant of Podman." style="width: 250px; padding: 5px;">
            <p style="text-align: center; padding-top: 0px"><i>Figure 1.9 Podman fork/exec architecture. The user launches Podman, which executes the OCI runtime, which then launches the container. The container is a direct descendant of Podman.</i></p>
        </div>
    </div>

Imagine you have a web service that you want to run at boot time. The web service is packaged in a container, so you need a container engine. In the **Docker case**, you need to 
- set it up to be running on your machine with each of the daemons running and accepting connections,
- launch the Docker client to start the web service. 
- Now you have your containerized application running as well as all of the Docker daemons. 

In the **Podman case**, 
- use the Podman command to launch your container, and Podman will go away. 
- Your container will continue to run without the overhead of running the multiple daemons. 

Less overhead is incredibly popular on low-end machines like IOT devices and edge servers.

## 1.3.7 Integration with systemd

Systemd is the fundamental init system in the operating systems. The init process on a Linux system is the first process that is started by the kernel on boot. Therefore, the init system is the ancestor of all processes and can monitor them all. Podman wants to fully integrate the running of containers with the init system. Users want to use systemd to start and stop containers at boot time. Containers should do the following:

- Support systemd within a container,
- Support socket activation,
- Support systemd notifications that a containerized application is fully activated,
- Allow systemd to fully manage the cgroups and lifespan of a containerized application.

Basically, containers work as services in systemd unit files. Many developers want to run systemd within a container to run multiple system-defined services within a container.

However, the upstream Docker community disagrees with this and has denied all pull requests that attempt to integrate systemd into Docker. They believe Docker should manage the life cycle of the container, and they do not want to accommodate users who want to run systemd in a container (This is [not true](https://docs.docker.com/config/daemon/systemd/) as of January 2024 - VR).

The upstream Docker community believes the Docker daemon, as opposed to systemd, should be the controller of processes, it should manage the life cycle of containers, and it should start and stop them at boot time. The problem is there are many more features in systemd than in Docker, including startup ordering, socket activation, service ready notifications, and so on.

When Podman was designed, the developers wanted to make sure it fully integrated with systemd. When you run systemd inside a container, Podman sets up the container the way systemd expects and allows it to simply run as PID1 of the container with limited privileges. Podman allows you to run services within the container the same way they run on a system or in a VM: via systemd unit files. Podman supports socket activation, service notifications, and many other systemd unit file features. Podman makes it simple to generate systemd unit files with best practices for running containers within a systemd service. For more information, see **chapter 7** on systemd integration.

The [Containers project](https://github.com/containers) where Podman, container libraries, and other container management tools reside, wants to embrace all features of the operating system and fully integrate it. Chapter 7 explains Podman integration with systemd.

## 1.3.8 Pods

One advantage of Podman is described in its name. As mentioned earlier, Podman is actually short for **Pod Manager**. As the official Kubernetes documentation puts it, 

> “A **pod** (as in a pod of seals, hence the logo, or pea pod) is a group of one or more containers, with shared storage/network resources, and a specification for how to run the containers.” 

Podman works with 
- either a single container at a time, like Docker, 
- or it can manage groups of containers together in a pod. 

One of the design goals of containers is 

> to separate services into single containers: **microservices**. Then you combine containers together to build larger services. 

Pods allow you to group multiple services together to form a larger service managed as a single entity. One of the goals of Podman is allowing you to experiment with pods. Figure 1.12 shows two pods running on a system, each pod containing three containers.

Podman has the `podman generate kube` command, which allows you to generate Kubernetes YAML files from running containers and pods, as you can see in chapter 7. Similarly, it has the `podman play kube` command, which allows you to play Kubernetes YAML files and generate pods and containers on your host. 

> I suggest using Podman for running pods and containers on a single host and using Kubernetes to take your pods and containers and run them on multiple machines and all through your infrastructure. 

Other projects, like [kind](https://kind.sigs.k8s.io/docs/user/rootless), are experimenting with running pods with Podman under the guidance of Kubernetes.

<center>
    <img src="../data/images/Screenshot_20240114_022123.png" alt="Figure 1.12 Two pods running on a host. Each pod runs two different containers along with the infra container." style="width: 650px">
    <p style="text-align: center"><i>Figure 1.12 Two pods running on a host. Each pod runs two different containers along with the infra container.</i></p>

## 1.3.11 Complete customizability

Container engines tend to have lots of built-in constants, like the namespaces they run with, whether or not SELinux is enabled, and which capabilities containers run with. With Docker, most of these values are hardcoded and cannot be changed by default. Podman, on the other hand, has a very customizable configuration.

Podman has its built-in defaults but defines three locations for its configuration files to be stored:

- `/usr/share/containers/containers.conf` — Where a distribution can define the changes the distribution likes to use
- `/etc/containers/containers.conf` — Where they can set up system overrides
- `$HOME/.config/containers/containers.conf` — Can be specified only in rootless mode

The configuration files allow you to configure Podman to run the way you want by default. You can even run with more security by default if you choose.

## 1.3.12 User-namespace support

Podman is fully integrated with the user namespace. Rootless mode relies on user namespaces, which allows for multiple UIDs to be assigned to a user. 

> ChatGPT: The passage is explaining that rootless mode relies on user namespaces, which is a feature in the Linux kernel that allows for the mapping of multiple UIDs (User IDs) to a single user. This means that a non-root user can have multiple UIDs assigned to them within a container, allowing them to access resources and perform actions as if they were a different user, without requiring actual root privileges on the host system. This feature is a key aspect of how Podman provides containerization without relying on the root user, enhancing security and minimizing the risk of security vulnerabilities.

User namespaces provide isolation between users on a system, so you can have multiple rootless users running containers with multiple user IDs, all isolated from each other.

A user namespace can be used to isolate containers from each other. Podman makes it simple to launch multiple containers, each with a unique user namespace. The kernel then isolates the processes from host users as well as each other based on UID separation.

Docker only supports running containers in a single, separate, user namespace, meaning all containers run within the same user namespace. Root in one container is the same as root in another container. It does not support running each container in a different user namespace, which means containers attack each other from a user-namespace perspective. Even though Docker supports this mode, almost no one runs containers with Docker in a separate user namespace (due to various reasons such as complexity and compatibility with existing systems - ChatGPT).

# <b>2. Podman command line</b>

- The Podman command line
- Running an OCI application
- Comparing containers and images
- Building an OCI-based image

In [4]:
man podman

podman(1)                   General Commands Manual                  podman(1)

NAME
       podman - Simple management tool for pods, containers and images

SYNOPSIS
       podman [options] command

DESCRIPTION
       Podman  (Pod  Manager)  is  a fully featured container engine that is a
       simple daemonless tool.  Podman provides a Docker-CLI  comparable  com‐
       mand  line  that  eases the transition from other container engines and
       allows the management of pods,  containers  and  images.   Simply  put:
       alias  docker=podman.   Most  Podman  commands  can be run as a regular
       user, without requiring additional privileges.

       Podman uses Buildah(1) internally  to  create  container  images.  Both
       tools share image (not container) storage, hence each can use or manip‐
       ulate images (but not containers) created by the other.

       Default settings for flags are defined in  containers.conf.  Most  set‐
       tings  for  Remote connections 

In [5]:
podman --help

Manage pods, containers and images

Usage:
  podman [options] [command]

Available Commands:
  attach      Attach to a running container
  auto-update Auto update containers according to their auto-update policy
  build       Build an image using instructions from Containerfiles
  commit      Create new image based on the changed container
  container   Manage containers
  cp          Copy files/folders between a container and the local filesystem
  create      Create but do not start a container
  diff        Display the changes to the object's file system
  events      Show podman events
  exec        Run a process in a running container
  export      Export container's filesystem contents as a tar archive
  generate    Generate structured data based on containers, pods or volumes
  healthcheck Manage health checks on containers
  help        Help about any command
  history     Show history of a specified image
  image       Manage images
  images      List images in local storage
 

# 2.1 `podman container`

In [2]:
man podman container

podman-container(1)         General Commands Manual        podman-container(1)

NAME
       podman-container - Manage containers

SYNOPSIS
       podman container subcommand

DESCRIPTION
       The container command allows you to manage containers

COMMANDS
       ┌───────────┬────────────────────────────────┬────────────────────────────────┐
       │Command    │ Man Page                       │ Description                    │
       ├───────────┼────────────────────────────────┼────────────────────────────────┤
       │attach     │ podman-attach(1)               │ Attach to a running container. │
       ├───────────┼────────────────────────────────┼────────────────────────────────┤
       │checkpoint │ podman-container-checkpoint(1) │ Checkpoints  one  or more run‐ │
       │           │                                │ ning containers.               │
       ├───────────┼────────────────────────────────┼────────────────────────────────┤
       │cleanup    │ podman-container-cleanup(

Developers, administrators, quality engineers, and general users primarily use the `podman run` command to pull down and run, test, or explore these container images. To start building out containerized applications, the first thing you need to do is start working with a **base image**. In our examples, you pull and run the [registry.access.redhat.com/ubi8/httpd-24](registry.access.redhat.com/ubi8/httpd-24) image to container storage in your home directory and start exploring inside the container.

## Exploring containers

```sh
podman run -ti --rm registry.access.redhat.com/ubi8/httpd-24 bash
```
```
Trying to pull registry.access.redhat.com/ubi8/httpd-24:latest...
Getting image source signatures
Copying blob 89e0ad8acaf1 done  
Copying blob c2650fe947f6 done  
Copying blob 50ccdb01751a done  
Copying config 203593be2e done  
Writing manifest to image destination
Storing signatures
```
```sh
bash-4.4$ 
```

By default the podman run command executes the containerized command in the foreground until the container exits. In this case, you end up at a Bash prompt running within the container and showing the `bash-4.4$` prompt. When you exit this Bash prompt, Podman stops the container.

- You used two options: `-t` and `-i`, as `-ti`, which tells Podman to hook up to the terminal. This connects to the input, output, and error stream of the bash process within the container to your screen, which allows you to interact within the container.

- The `--rm` option tells Podman to delete the container as soon as the container exits, freeing up all of the container’s storage.

- Specify the container image, [registry.access.redhat.com/ubi8/httpd-24](registry.access.redhat.com/ubi8/httpd-24), you are working with. The `podman` command reaches out to the container registry at `registry.access.redhat.com` and begins copying down the `ubi8/httpd-24:latest` image. Podman copies multiple **layers** (aka **blobs**), as shown in the following listing, and stores them in the local container storage (**container host**). You see the progress as the image layers are pulled down. Some images are rather large and can take a long time while being pulled down. If you later run a different container on the same image, Podman skips the image-pulling step, since you already have the correct image in local container storage.

- Finally, specify the executable to be run within the container, in this case, `bash`.

While inside the bash shell container, `cat /etc/os-release`, and notice it is likely a different OS or a different version than the `/etc/os-release` outside the container. Explore around in the container, and notice how different it is from your host environment:

```sh
bash-4.4$ grep PRETTY_NAME /etc/os-release 
```
```
PRETTY_NAME="Red Hat Enterprise Linux 8.9 (Ootpa)"
```

```sh
bash-4.4$ ls /usr/bin | wc -l    # commands available
```
```
526
```
```sh
bash-4.4$ ps
```
```
    PID TTY          TIME CMD
      1 pts/0    00:00:00 bash
      9 pts/0    00:00:00 ps
```

You can further explore the inside of the container to gain an understanding of what is going on within a container.

When you are done, you exit the bash script, and the container shuts down. Since you ran with the `--rm` option, Podman removes all the container storage and deletes the container. The container image remains in `containers/storage`:

```sh
$ sudo ls /var/lib/containers/storage/
defaultNetworkBackend  libpod  mounts  overlay  overlay-containers  overlay-images  overlay-layers  storage.lock  tmp  userns.lock

$ sudo tree /var/lib/containers/storage/
/var/lib/containers/storage/
├── defaultNetworkBackend
├── libpod
│   └── bolt_state.db
├── mounts
├── overlay
│   └── l
├── overlay-containers
│   └── containers.lock
├── overlay-images
│   └── images.lock
├── overlay-layers
│   └── layers.lock
├── storage.lock
├── tmp
└── userns.lock

9 directories, 7 files
```

## Running the containerized application

### `podman run`

```sh
$ podman run -d -p 8080:8080 --name myapp registry.access.redhat.com/ubi8/httpd-24
37a1d2e31dbf4fa311a5ca6453f53106eaae2d8b9b9da264015cc3f8864fac22
```

First, remove the `-ti` and the `--rm` options, since you want the container to remain running when the podman command exits. You are not a shell running within the container interactively, since it is just running the containerized web service:

- `-d` (`--detach`) option tells Podman to launch the container and then detach from it. Basically, run the container in the background. The Podman command actually exits and leaves the container running. Chapter 6 goes much deeper into what is going on behind the scenes;

- `-p` (`--publish`) option tells Podman to publish or bind the container port `8080` to the host port `8080` when the container is running. With the `-p` option, the field before the colon refers to the host port, while the field after the colon refers to the container port. In this case, you see that the ports are the same. If you specify only one port, Podman considers this port a container port and randomly picks a host port on which the container port is bound. You can use the `podman port` command to discover which ports are bound to a container;

```sh
- $ podman port myapp
8080/tcp -> 0.0.0.0:8080
```

i.e. the port `8080/tcp` inside the container is bound to all of the host networks (`0.0.0.0`) at port `8080`.

By default, containers are created within _their own_ **network namespace**, meaning they are not bound to the host network but to their virtualized network. Suppose I execute the container without the `-p` option. In that case, the Apache server within the container binds to the network interface within the container’s network namespace, but Apache is not bound to the host network.

Only processes within the container are able to connect to port `8080` to communicate with the web server. By executing the command with the `-p` option, Podman connects the port from inside the container to the host network at the specified port. The connection allows external processes like a web browser to read from the web service.

> NOTE: If you are running containers in rootless mode, covered in **chapter 3**, Podman users are by default not permitted to bind to ports `< 1024` by the kernel. Some containers want to bind to lower ports like port `80`, which is allowed inside the container, but `-p 80:80` fails, since `80` is less than `1024`. Using `-p 8080:80` causes Podman to bind the host’s port `8080` to port `80` within the container. The [upstream Podman repo](http://mng.bz/69ry) contains troubleshooting information on problems like binding to ports less than `1024` and many others.

The `-p` option can map port numbers inside the container to different port numbers outside the container.

- `--name myapp` option. Specifying a name makes it easier to find the container, and it allows you to specify a name that can then be used for other commands (e.g., `podman stop myapp`). If you don’t specify a name, Podman automatically generates a unique container name along with a container ID. All of the Podman commands that interact with containers can use either the name or the ID.

When the `podman run` command completes, the container is running. Since this container is running in detached mode, Podman prints out the container ID and exits, but the container remains running.

Now that the container is running, you can launch a web browser to communicate with the web server inside of the container at localhost port `8080` (see figure 2.1):

![Figure 2.1 Web browser window connecting to the ubi8/httpd-24 container running in Podman](../data/images/Screenshot_20240118_004318.png)

Now imagine you want to start another container. You can execute a similar command with just a couple of changes:

```sh
$ podman run -d -p 8081:8080 --name myapp1 registry.access.redhat.com/ubi8/httpd-24
fa41173e4568a8fa588690d3177150a454c63b53bdfa52865b5f8f7e4d7de1e1
```

Notice you need to change the name of the container to `myapp1`; otherwise, the `podman run command` fails with the `myapp` name because the container previously existed. Also you need to change the `-p` option to use `8081` for the host port because the previous container, myapp, is currently running and is bound to port `8080`. The second container isn’t allowed to bind to port 8080 until the first container exits.

Some notable `podman run` options include the following:

- `--user USERNAME` — This tells Podman to run the container as a specific user defined in the image. By default, Podman will run the container as `root`, unless the container image specifies a default user.
- `--rm` — This automatically removes the container when it exits.
- `--tty` (`-t`) — This allocates a pseudo `-tty` and attaches it to the standard input of the container.
- `--interactive` (`-i`) — This connects stdin to the primary process of the container. These options give you an interactive shell within the container.

> NOTE There are dozens of `podman run` options available, allowing you to change security features, namespaces, volumes, and so on. Some of these I use and explain throughout the book. Refer to the `podman-run` man page for a description of all of the options. Most of the `podman create` options defined in table 2.1 are also available for `podman run`.

Use the `man podman-run` command for information about all options. Now that the container is up and running, it is time to stop the container and go to the next step.

In [1]:
man podman-run

podman-run(1)               General Commands Manual              podman-run(1)

NAME
       podman-run - Run a command in a new container

SYNOPSIS
       podman run [options] image [command [arg ...]]

       podman container run [options] image [command [arg ...]]

DESCRIPTION
       Run  a process in a new container. podman run starts a process with its
       own file system, its own networking, and its own isolated process tree.
       The  image  which starts the process may define defaults related to the
       process that will be run in the container, the  networking  to  expose,
       and  more, but podman run gives final control to the operator or admin‐
       istrator who starts the container from the image. For that reason  pod‐
       man run has more options than any other Podman command.

       If the image is not already loaded then podman run will pull the image,
       and all image dependencies, from the repository in the same way running
       podman pull image

### `podman create`

The `podman create` command is almost identical to the `podman run` command. The `create` command pulls the image if it is not in container storage and configures the container information to make it ready to run but never executes the container. It is often used together with the `podman start` command described in section 2.1.4. You might want to create a container and then later use a systemd unit file to start and stop the container.

In [3]:
man podman create

podman-create(1)            General Commands Manual           podman-create(1)

NAME
       podman-create - Create a new container

SYNOPSIS
       podman create [options] image [command [arg ...]]

       podman container create [options] image [command [arg ...]]

DESCRIPTION
       Creates  a  writable  container layer over the specified image and pre‐
       pares it for running the specified command. The container  ID  is  then
       printed  to  STDOUT.  This  is similar to podman run -d except the con‐
       tainer is never started. You can then use the  podman  start  container
       command to start the container at any point.

       The initial status of the container created with podman create is 'cre‐
       ated'.

       Default settings for flags are defined in  containers.conf.  Most  set‐
       tings  for  remote connections use the server's containers.conf, except
       when documented in man pages.

IMAGE
       The image is specified using transport:path forma

### `podman stop`

```sh
$ podman stop myapp
```

You have two containers running and have tested them by running a web browser against them. To continue the development by actually adding some content to the web page, you can stop the containers using the `podman stop` command:

The `stop` command stops the container started with the previous `podman run` command.

When stopping a container, Podman examines the running container and sends a stop signal, usually `SIGTERM`, to the primary process (`PID1`) of the container, and then by default it waits `10` seconds for the container to stop. The stop signal tells the primary process within the container to exit gracefully. If the container doesn’t stop within `10` seconds, Podman sends the `SIGKILL` signal to the process, forcing the container to stop. The 10-second wait gives the processes in the container time to clean up and commit changes.

The default stop signal can be changed for a container using the `podman run --stop-signal` option. Sometimes the primary or init process of a container ignores `SIGTERM` (e.g., containers that use systemd as the primary process inside a container). systemd ignores `SIGTERM` and specifies that it shuts down using the `SIGRTMIN+3 `(`signal #37`) signal. The stop signal can be embedded in container images, as I describe in section 2.3.

Some containers ignore the `SIGTERM` stop signal, which means you have to wait 10 seconds for the container to exit. If you know the container ignores the default stop signal, and you don’t care about the container cleaning up, you can just add the `-t 0` option to podman stop to send the `SIGKILL` signal right away:

```sh
$ podman stop -t 0 myapp1
myapp1
```

Some notable `Podman stop` options include the following:

- `--time` (`-t`) — This sets the timeout; `-t 0` sends the `SIGKILL` without waiting for the container to stop.
- `--latest` (`-l`) — This is a useful option to allow you to stop the last created container rather than having to use the container name or container ID. Most Podman commands that require you to specify a container name or ID also accept the `--latest` option. This is only available on Linux machines.
- `--all` — This tells Podman to stop all running containers. Similarly to `--latest`, Podman commands that require a container name or container ID parameter also take the `--all` option.

Use the `man podman-stop` command for information about all options.

In [6]:
man podman-stop

podman-stop(1)              General Commands Manual             podman-stop(1)

NAME
       podman-stop - Stop one or more running containers

SYNOPSIS
       podman stop [options] container ...

       podman container stop [options] container ...

DESCRIPTION
       Stops  one  or  more containers.  You may use container IDs or names as
       input. The --time switch allows you to specify the number of seconds to
       wait  before  forcibly stopping the container after the stop command is
       issued to the container. The default is 10 seconds.  By  default,  con‐
       tainers  are  stopped  with SIGTERM and then SIGKILL after the timeout.
       The SIGTERM default can be overridden by the image used to  create  the
       container and also via command line when creating the container.

OPTIONS
   --all, -a
       Stop all running containers.  This does not include paused containers.

   --cidfile=file
       Read  container ID from the specified file and stop the container.

### `podman kill`

Podman has a similar command, `podman kill`, which sends the specified kill signal. The `podman kill` command can be useful when you want to send signals into the container without actually stopping the container.

In [2]:
man podman kill

podman-kill(1)              General Commands Manual             podman-kill(1)

NAME
       podman-kill - Kill the main process in one or more containers

SYNOPSIS
       podman kill [options] [container ...]

       podman container kill [options] [container ...]

DESCRIPTION
       The  main process inside each container specified will be sent SIGKILL,
       or any signal specified with option --signal.

OPTIONS
   --all, -a
       Signal all running and paused containers.

   --cidfile=file
       Read container ID from the specified file and kill the container.   Can
       be specified multiple times.

   --latest, -l
       Instead  of  providing  the  container name or ID, use the last created
       container. If you use methods other than Podman to run containers  such
       as  CRI-O,  the  last  started  container could be from either of those
       methods. (This option is not available with the remote  Podman  client,
       including Mac and Windows (excluding WSL2) ma

## `podman start`

Eventually, your system will have lots of stopped containers, and sometimes you will need to restart them (e.g., if the system was rebooted). Another common use case is to first `create` a container and later `start` it.

The container you created has now been stopped. Next, you may want to start it back up again using the command in the following listing:

```sh
$ podman start myapp
myapp
```

The `podman start` command starts one or more containers. This command will output the container ID, indicating that your container is up and running. You can now reconnect to it with a web browser. One common use case for `podman start` is starting a container after a reboot to start all of the containers that were stopped during shutdown.

Some favorite Podman start options include these:

- `--all` — This starts all of the stopped containers in container storage.
- `--attach` — This attaches your terminal to the output of the container.
- `--interactive` (`-i`)—This attaches the terminal input to the container.

Use the `man podman-start` command for information about all options.

In [3]:
man podman-start

podman-start(1)             General Commands Manual            podman-start(1)

NAME
       podman-start - Start one or more containers

SYNOPSIS
       podman start [options] container ...

       podman container start [options] container ...

DESCRIPTION
       Start  one  or  more containers.  You may use container IDs or names as
       input.  The attach and interactive options cannot be used  to  override
       the  --tty  and  --interactive options from when the container was cre‐
       ated. If you attempt to start a running container with the --attach op‐
       tion, podman will simply attach to the container.

OPTIONS
   --all
       Start  all  the  containers  created by Podman, default is only running
       containers.

   --attach, -a
       Attach container's STDOUT and STDERR.  The default is false.  This  op‐
       tion cannot be used when starting multiple containers.

   --detach-keys=sequence
       Specify  the key sequence for detaching a container. Format i

## `podman ps` | Listing containers

After you’ve been using Podman for a while and have pulled down and run many different container images, you might want to figure out which containers are running or which containers you have in local storage. You will need to be able to list these containers.

Use the `podman ps` command to list containers:

```sh
$ podman ps
```
```
CONTAINER ID IMAGE                      COMMAND        CREATED \ 
➥   STATUS         PORTS          NAMES
b1255e94d084 registry.access.redhat.com/ubi8/httpd-24:latest /usr/bin/run-\ 
➥ http... 6 minutes ago Up 4 minutes ago 0.0.0.0:8080->8080/tcp myapp
```

Notice the `podman ps` command by default lists the running containers. Use the `--all` option to see all of the containers:

```sh
$ podman ps --all
```
```
CONTAINER ID IMAGE                       COMMAND        CREATED \ 
➥   STATUS          PORTS          NAMES
b1255e94d084 registry.access.redhat.com/ubi8/httpd-24:latest /usr/bin/run-\
➥ http... 9 minutes ago Up 8 minutes ago     0.0.0.0:8080->8080/tcp myapp
3efee4d39965 registry.access.redhat.com/ubi8/httpd-24:latest /usr/bin/run-\
➥ http... 7 minutes ago Exited (0) 3 minutes ago 0.0.0.0:8081->8080/tcp myapp1
```

Some notable `podman ps` options include the following:

- `--all` — This tells Podman to list all containers rather than just running containers.
- `--quiet` — This tells Podman to only print the container IDs.
- `--size` — This tells Podman to return the amount of disk space currently used for each container other than the images they are based on.

Use the `man podman-ps` command for information about all options. 

In [4]:
man podman-ps

podman-ps(1)                General Commands Manual               podman-ps(1)

NAME
       podman-ps - Prints out information about containers

SYNOPSIS
       podman ps [options]

       podman container ps [options]

       podman container list [options]

       podman container ls [options]

DESCRIPTION
       podman  ps  lists  the  running containers on the system. Use the --all
       flag to view all the containers information.  By default it lists:

              • container id

              • the name of the image the container is using

              • the COMMAND the container is executing

              • the time the container was created

              • the status of the container

              • port mappings the container is using

              • alternative names for the container

OPTIONS
   --all, -a
       Show all the containers created by Podman, default is only running con‐
       tainers.

       Note: Podman shares containers storage with other tools such

## `podman inspect` | Inspecting containers

Also see [Skopeo](#Skopeo).

The `podman ps` command gives us some data about the containers, but if you want to really examine information about the container, you can use the `podman inspect` command.

It can also be used to 
- inspect images, 
- networks, 
- volumes, and 
- pods. 

The `podman container inspect` command is also available and specific to containers. But most users just type the shorter `podman inspect` command:

```sh
podman run -d -p 8080:8080 --name myapp registry.access.redhat.com/ubi8/httpd-24
```
```
Trying to pull registry.access.redhat.com/ubi8/httpd-24:latest...
Getting image source signatures
Copying blob 50ccdb01751a done  
Copying blob c2650fe947f6 done  
Copying blob 89e0ad8acaf1 done  
Copying config 203593be2e done  
Writing manifest to image destination
Storing signatures
b07f2128bf2efea081cbb7bfc6e7399500097f64c2350df146ed67420ecdf1e4
```

In [2]:
podman inspect myapp | jq

[
  {
    "Id": "b07f2128bf2efea081cbb7bfc6e7399500097f64c2350df146ed67420ecdf1e4",
    "Created": "2024-01-23T02:38:23.543125296+05:00",
    "Path": "container-entrypoint",
    "Args": [
      "/usr/bin/run-httpd"
    ],
    "State": {
      "OciVersion": "1.0.2-dev",
      "Status": "running",
      "Running": true,
      "Paused": false,
      "Restarting": false,
      "OOMKilled": false,
      "Dead": false,
      "Pid": 15546,
      "ConmonPid": 15544,
      "ExitCode": 0,
      "Error": "",
      "StartedAt": "2024-01-23T02:38:25.611986382+05:00",
      "FinishedAt": "0001-01-01T00:00:00Z",
      "Health": {
        "Status": "",
        "FailingStreak": 0,
        "Log": null
      },
      "CgroupPath": "/user.slice/user-1000.slice/user@1000.service/user.slice/libpod-b07f2128bf2efea081cbb7bfc6e7399500097f64c2350df146ed67420ecdf1e4.scope",
      "CheckpointedAt": "0001-01-01T00:00:00Z",
      "RestoredAt": "0001-01-01T00:00:00Z"
    },
    "Image": "203593be2e5cc693e0dcbffc46d6

In [10]:
# podman container inspect myapp | jq

In [3]:
podman inspect myapp | wc -l

288


As you can see, the `podman inspect` command outputs a large JSON file — 307 lines on my machine. All of this information is eventually handed down the OCI runtime to launch the container. When using the `inspect` command, it is often better to pipe its output to `less` or `grep` to find particular fields you are interested in. 

```sh
podman inspect myapp | less
```

Alternatively, you can use the `--format` option. If you want to examine the command executed when you start the container, execute the following:

In [4]:
podman inspect --format '{{ .Config.Cmd }}' myapp

[/usr/bin/run-httpd]


Or if you want to see the `stop signal` (The default stop signal for all containers is `15` - `SIGTERM`), execute the following:

In [5]:
podman inspect --format '{{ .Config.StopSignal }}' myapp

15


Some notable podman inspect options include the following:

- `--latest` (`-l`) — quickly inspect the last created container rather than specifying the container name or container ID.
- `--format` — extract particular fields out of the JSON.
- `--size` — the amount of disk space the container is using. Gathering this information takes a long time, so it is not done by default.

### mans

In [6]:
man podman inspect

podman-inspect(1)           General Commands Manual          podman-inspect(1)

NAME
       podman-inspect  - Display a container, image, volume, network, or pod's
       configuration

SYNOPSIS
       podman inspect [options] name [...]

DESCRIPTION
       This displays the low-level information on containers and images  iden‐
       tified  by  name  or  ID. By default, this will render all results in a
       JSON array. If the inspect type is all, the  order  of  inspection  is:
       containers, images, volumes, network, pods.
        So,  if  a container has the same name as an image, then the container
       JSON will be returned, and so on.
        If a format is specified, the given template will be executed for each
       result.

       For more inspection options, see also podman-container-inspect(1), pod‐
       man-image-inspect(1), podman-network-inspect(1), podman-pod-inspect(1),
       and podman-volume-inspect(1).

OPTIONS
   --format, -f=format
       Format  the 

In [7]:
man podman-container-inspect

podman-container-inspect(1) General Commands Manualpodman-container-inspect(1)

NAME
       podman-container-inspect - Display a container's configuration

SYNOPSIS
       podman container inspect [options] container [container ...]

DESCRIPTION
       This  displays  the  low-level  information on containers identified by
       name or ID. By default, this will render all results in a  JSON  array.
       If  a format is specified, the given template will be executed for each
       result.

OPTIONS
   --format, -f=format
       Format the output using the given Go template.  The  keys  of  the  re‐
       turned  JSON can be used as the values for the --format flag (see exam‐
       ples below).

       Valid placeholders for the Go template are listed below:

       ┌─────────────────────┬───────────────────────────────┐
       │Placeholder          │ Description                   │
       ├─────────────────────┼───────────────────────────────┤
       │.AppArmorProfile     │ AppArm

## `podman rm` | Removing containers

If you are done using a container, you may want to remove the container to free up disk space or reuse the container name. Make sure to stop the container (section 2.1.3) before removing it. Then use the podman `rm` command to remove container:

In [12]:
podman ps

CONTAINER ID  IMAGE                                            COMMAND               CREATED         STATUS             PORTS                   NAMES
b07f2128bf2e  registry.access.redhat.com/ubi8/httpd-24:latest  /usr/bin/run-http...  16 minutes ago  Up 16 minutes ago  0.0.0.0:8080->8080/tcp  myapp


In [13]:
podman stop myapp

myapp


In [14]:
podman rm myapp

myapp


In [15]:
podman ps

CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES


Some notable `podman rm` options include the following:

- `--all` — This option is useful if you want to remove all your containers.
- `--force` — This option tells Podman to stop all the running containers when removing.

In [16]:
man podman rm

podman-rm(1)                General Commands Manual               podman-rm(1)

NAME
       podman-rm - Remove one or more containers

SYNOPSIS
       podman rm [options] container

       podman container rm [options] container

DESCRIPTION
       podman  rm  will remove one or more containers from the host.  The con‐
       tainer name or ID can be used.  This does not remove  images.   Running
       or unusable containers will not be removed without the -f option.

OPTIONS
   --all, -a
       Remove all containers.  Can be used in conjunction with -f as well.

   --cidfile=file
       Read container ID from the specified file and rm the container.  Can be
       specified multiple times.

   --depend
       Remove selected container and recursively remove  all  containers  that
       depend on it.

   --filter=filter
       Filter what containers remove.  Multiple filters can be given with mul‐
       tiple uses of the --filter flag.  Filters with the same key work inclu‐
       s

In [26]:
man podman stop

podman-stop(1)              General Commands Manual             podman-stop(1)

NAME
       podman-stop - Stop one or more running containers

SYNOPSIS
       podman stop [options] container ...

       podman container stop [options] container ...

DESCRIPTION
       Stops  one  or  more containers.  You may use container IDs or names as
       input. The --time switch allows you to specify the number of seconds to
       wait  before  forcibly stopping the container after the stop command is
       issued to the container. The default is 10 seconds.  By  default,  con‐
       tainers  are  stopped  with SIGTERM and then SIGKILL after the timeout.
       The SIGTERM default can be overridden by the image used to  create  the
       container and also via command line when creating the container.

OPTIONS
   --all, -a
       Stop all running containers.  This does not include paused containers.

   --cidfile=file
       Read  container ID from the specified file and stop the container.

## `exec`-ing into a container

Often, when a container is running, you might want to start another process within the container to debug or examine what is going on. In some cases, you may want to modify some of the content the container is using.

Imagine you want to go into your container and modify the web page it is showing. You can `exec` into the container using the `podman exec` command. Use the `--interactive` (`-i`) option to allow you to execute commands within the container. You need to specify the name of the container `myapp` and execute the Bash script while in the container. If you stopped the `myapp` container, you need to restart it, since `podman exec` only works on running containers.

In the following example, you will `exec` a bash process into the container to create the `/var/www/html/index.html` file. You will write HTML content that causes the containerized website to display "Hello World".

Create a new container, as we have just removed it:

```sh
podman run -d -p 8080:8080 --name myapp registry.access.redhat.com/ubi8/httpd-24
```

In [17]:
podman ps

CONTAINER ID  IMAGE                                            COMMAND               CREATED         STATUS             PORTS                   NAMES
15f2d32fb930  registry.access.redhat.com/ubi8/httpd-24:latest  /usr/bin/run-http...  45 seconds ago  Up 42 seconds ago  0.0.0.0:8080->8080/tcp  myapp


In [2]:
podman exec -i myapp bash -c 'cat > /var/www/html/index.html' << _EOF
<html>
 <head>
 </head>
 <body>
  <h1>Hello World</h1>
 </body>
</html>
_EOF

`exec`-ing back into the container a second time, you can see that the file was successfully modified. This shows that modifications to a container via `exec` are permanent to the container and will remain even if you stopped and restarted the container. A key difference between `podman run` and `podman exec` is that 
- `run` creates a new container off of an image with processes running inside, while 
- `exec` starts processes inside of existing containers:

Now let’s connect a web browser to the container to see if the content has changed:

![](../data/images/Screenshot_20240123_031025.png)

Some notable `podman exec` options include the following:

- `--tty` — This connects a `-tty` to the exec session.
- `--interactive` — The `-i` option tells Podman to run in interactive mode, meaning you can interact with an exec-ed program, like a shell.

In [25]:
podman stop myapp
podman rm myapp

myapp
myapp


In [26]:
podman ps

CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES


In [24]:
man podman exec

podman-exec(1)              General Commands Manual             podman-exec(1)

NAME
       podman-exec - Execute a command in a running container

SYNOPSIS
       podman exec [options] container [command [arg ...]]

       podman container exec [options] container [command [arg ...]]

DESCRIPTION
       podman exec executes a command in a running container.

OPTIONS
   --detach, -d
       Start  the  exec session, but do not attach to it. The command will run
       in the background and the exec session will  be  automatically  removed
       when  it  completes.  The  podman exec command will print the ID of the
       exec session and exit immediately after it starts.

   --detach-keys=sequence
       Specify the key sequence for detaching a container. Format is a  single
       character [a-Z] or one or more ctrl-<value> characters where <value> is
       one of: a-z, @, ^, [, , or _. Specifying "" will disable this  feature.
       The default is ctrl-p,ctrl-q.

       This optio

## `podman commit` | Create your own image

Developers often run containers from a base image to create a new container environment. Once they are done, they pack this environment into a container image to be able to share it with other users. Those users can then use Podman to launch the containerized application. You can do this with Podman by committing the container to an OCI image.

```sh
$ podman commit myapp myimage
```
```
WARN[0000] archive: skipping "/home/commi/.local/share/containers/storage/vfs/dir/56afea0efea2ac0eb164154c366246b5adfe57d1d54d72c212e5000477e52ad9/run/httpd/cgisock.1" since it is a socket 
Getting image source signatures
Copying blob 86426b9e591d skipped: already exists  
Copying blob ed3888a8ac3e skipped: already exists  
Copying blob 9859219993e6 skipped: already exists  
Copying blob 7c6a6b5f8096 done  
Copying config 3d733e82ab done  
Writing manifest to image destination
Storing signatures
3d733e82ab2b67409b2fa956eeedb5869e468a12bd93d98980ed6a2307d30965
```

Now you can create a new container based on `myimage`:

```sh
$ podman run -d --name myapp1 -p 8081:8080 myimage
```
```
dbe6842306071232367f5b86da94dc0b09e487b17152a29a77e2d1900142fee4
```

> NOTE: Using the `podman commit` command to create an image is not a common method. The entire process of building container images can be scripted and automated using `podman build`. See section 2.3 for more information on this process.

In [3]:
podman images

REPOSITORY                                TAG         IMAGE ID      CREATED         SIZE
localhost/myimage                         latest      3d733e82ab2b  19 minutes ago  459 MB
registry.access.redhat.com/ubi8/httpd-24  latest      203593be2e5c  3 weeks ago     458 MB


### `--squash`

_ChatGPT:_  
The `podman commit --squash` command is used to create a new image based on a container's changes and layers, while also collapsing all the layers into a single layer to reduce the size of the resulting image. When a container is running and changes are made inside it, multiple new layers are created to represent those changes. When you use the `podman commit` command without the `--squash` flag, all of these layers are kept, which can result in a larger and less efficient image.

By using the `podman commit --squash` command, you can collapse all those layers into a single layer, making the resulting image smaller and more efficient. This can be useful for reducing the size of images before pushing them to a registry or sharing them with others.

However, it's important to consider some potential drawbacks of using the `--squash` flag. By collapsing all the layers into a single layer, you lose the ability to inspect or revert to individual changes made in the container. This can make troubleshooting and debugging more difficult. Additionally, squash operations can be resource-intensive and time-consuming, especially for large containers with many layers.

In general, using the `podman commit --squash` command can be beneficial for reducing the size of images, especially for production use. However, it's important to weigh the trade-offs and consider whether the loss of individual layer information is acceptable for your specific use case.

In [1]:
man podman commit

podman-commit(1)            General Commands Manual           podman-commit(1)

NAME
       podman-commit - Create new image based on the changed container

SYNOPSIS
       podman commit [options] container [image]

       podman container commit [options] container [image]

DESCRIPTION
       podman commit creates an image based on a changed container. The author
       of the image can be set using the --author OPTION.  Various  image  in‐
       structions can be configured with the --change OPTION and a commit mes‐
       sage can be set using the --message OPTION. The container and its  pro‐
       cesses  are  paused  while  the  image is committed. This minimizes the
       likelihood of data corruption when creating the new image. If  this  is
       not desired, the --pause OPTION can be set to false. When the commit is
       complete, Podman will print out the ID of the new image.

       If image does not begin with a registry name component, localhost  will
       be  adde

# 2.2 `podman image` | Manage images

In [12]:
man podman image

podman-image(1)             General Commands Manual            podman-image(1)

NAME
       podman-image - Manage images

SYNOPSIS
       podman image subcommand

DESCRIPTION
       The image command allows you to manage images

COMMANDS
       ┌────────┬─────────────────────────┬────────────────────────────────┐
       │Command │ Man Page                │ Description                    │
       ├────────┼─────────────────────────┼────────────────────────────────┤
       │build   │ podman-build(1)         │ Build   a  container  using  a │
       │        │                         │ Dockerfile.                    │
       ├────────┼─────────────────────────┼────────────────────────────────┤
       │diff    │ podman-image-diff(1)    │ Inspect changes on an  image's │
       │        │                         │ filesystem.                    │
       ├────────┼─────────────────────────┼────────────────────────────────┤
       │exists  │ podman-image-exists(1)  │ Check  if  an  image exis

## Container vs Image

In the container world, there is no more overused term than **container**. Often container refers to the running processes launched by Podman. But container can also refer to container data as the non-running objects sitting in container storage. As you saw in the previous section, `podman ps --all` shows running and non-running containers.

Another example is the term **namespace**, which is used in many different ways. I often get confused when people talk about namespaces within Kubernetes. Some people hear the term and think of virtual clusters, but when I hear it I think of Linux namespaces used with Pods and Containers. Similarly, **image** can refer to a VM image, a container image, an OCI image, or a Docker image stored at a container registry.

I think of **containers** as executing processes within an environment or something that is being prepared to run. In contrast, **images** are committed containers, which are prepared to be shared with others. Other users or systems can use these images to create new containers.

**Container images** are just _committed_ containers. The OCI defines the format of an image. Podman uses the [containers/image library](https://github.com/containers/image) for all of its interaction with images. Container images can be stored in different types of storage or **transports**, as `container/image` refers to them. These transports can be **container registries**, Docker archives, OCI archives, docker-daemon, as well as `containers/storage`. See section 2.2.4 for more information on transports.

In the context of Podman, I usually refer to **images** as the content stored locally in a container storage or in container registries like `docker.io` and `quay.io`. Podman uses the GitHub [container/storage library](https://github.com/containers/storage) for handling locally stored images. Let’s take a closer look at it.

### `container/storage`

The `container/storage` library provides the concept of a **storage container**. Basically, storage containers are intermediate storage content that hasn’t been committed yet. Think of them as files on disk and some JSON describing the content. Podman has its own datastore of data related to a Podman container, and Podman needs to deal with multiple users of its containers at the same time. It relies on filesystem locking provided by `containers/storage` to make sure hundreds of Podman executables can reliably share the same datastore.

When you commit a container to storage, Podman copies the container storage to the **image storage**. Images are stored in a series of layers, with every commit creating a new layer.

In our previous example, you used the `ubi8/httpd-24` image, which is two layers: 
- the base layer is `ubi8`, and then 
- the image provided added the `httpd` package and a few others to create the `ubi8/httpd-24`. 

Now when you commit your container in the previous section, Podman adds another layer on top of the `ubi8/httpd-24` image called `myimage`.

Info from the `container/storage` library's github page:

The `container/storage` library manages three types of items: layers, images, and containers.

A **layer** is a copy-on-write filesystem which is notionally stored as a set of changes relative to its parent layer, if it has one. A given layer can only have one parent, but any layer can be the parent of multiple layers. Layers which are parents of other layers should be treated as read-only.

An **image** is a reference to a particular layer (its top layer), along with other information which the library can manage for the convenience of its caller. This information typically includes configuration templates for running a binary contained within the image's layers, and may include cryptographic signatures. Multiple images can reference the same layer, as the differences between two images may not be in their layer contents.

A **container** is a read-write layer which is a child of an image's top layer, along with information which the library can manage for the convenience of its caller. This information typically includes configuration information for running the specific container. Multiple containers can be derived from a single image.

### `podman image tree`

One handy Podman command for showing the layers of an image is the `podman image tree` command:

In [6]:
podman image tree myimage

Image ID: 3d733e82ab2b
Tags:     [localhost/myimage:latest]
Size:     458.5MB
Image Layers
├── ID: 86426b9e591d Size: 212.8MB
├── ID: 1c81eee24214 Size: 59.49MB
├── ID: 98072eee8668 Size: 186.2MB Top Layer of: [registry.access.redhat.com/ubi8/httpd-24:latest]
└── ID: 1e375fc5f3b8 Size: 50.69kB Top Layer of: [localhost/myimage:latest]



### `podman image diff`

Another useful Podman command, `podman image diff`, allows you to see the actual files and directories that have been 
- **`C`hanged**, 
- **`A`dded**, or 
- **`D`eleted**

compared to another image or the lower layer:

In [7]:
podman image diff myimage

C /opt
C /opt/app-root
C /opt/app-root/etc
A /opt/app-root/etc/passwd
C /etc
C /etc/group
C /etc/httpd
C /etc/httpd/conf
C /etc/httpd/conf/httpd.conf
C /etc/httpd/conf.d
C /etc/httpd/conf.d/ssl.conf
C /etc/httpd/tls
A /etc/httpd/tls/localhost.crt
A /etc/httpd/tls/localhost.key
A /etc/httpd/tls/dhparams.pem
C /run/httpd
A /run/httpd/httpd.pid
C /var
C /var/log
C /var/log/httpd
A /var/log/httpd/modsec_audit.log
A /var/log/httpd/modsec_debug.log


In [8]:
podman image diff myimage | grep A

A /run/httpd/httpd.pid
A /var/log/httpd/modsec_audit.log
A /var/log/httpd/modsec_debug.log
A /opt/app-root/etc/passwd
A /etc/httpd/tls/dhparams.pem
A /etc/httpd/tls/localhost.crt
A /etc/httpd/tls/localhost.key


Images are just TAR diffs of software applied on lower-level images, and container content is an uncommitted layer of software. Once a container is committed, you can create other containers on top of your image. You can also share the image with others, so they can create other containers on your image. Now let’s look at all the images in your container storage.

## `podman images` | Listing images

In the container section, you were working with images and used command `podman images` to list the images in local storage:

In [9]:
podman images

REPOSITORY                                TAG         IMAGE ID      CREATED      SIZE
localhost/myimage                         latest      3d733e82ab2b  3 hours ago  459 MB
registry.access.redhat.com/ubi8/httpd-24  latest      203593be2e5c  3 weeks ago  458 MB


</br>

<center>
Table 2.2 Default fields listed by the <code>podman images</code> command<br>
<br>
    
|Heading|Description|
|-|-|
|REPOSITORY|Complete name of the image|
|TAG|Version (tag) of the image. Image tagging is covered in section 2.2.6.|
|IMAGE ID|Unique identifier of the image. It is generated by Podman as a SHA256 hash of the image's JSON configuration object.|
|CREATED|Elapsed time since the image was created. Images are sorted by this field by default.|
|SIZE|The amount of storage being used by the image.|

> NOTE: Over time, the amount of storage used by all the images you pull grows. It is relatively common for users to run out of disk space, so you should monitor the size of images and containers, removing them when you are no longer using them. Use the `man podman-system-prune` command for more information on cleaning up.

One notable `podman image` option is the following:

- `--all` (`-a`) — This option is useful for listing all images. By default, `podman-images` lists only the images currently in use. When an image is replaced by a newer image with the same tag, the previous image is tagged as `<none><none>`; These images are called **dangling images**. I cover dangling images in section 2.3.1.

In [4]:
man podman images

podman-images(1)            General Commands Manual           podman-images(1)

NAME
       podman-images - List images in local storage

SYNOPSIS
       podman images [options] [image]

       podman image list [options] [image]

       podman image ls [options] [image]

DESCRIPTION
       Displays locally stored images, their names, and their IDs.

OPTIONS
   --all, -a
       Show  all images (by default filter out the intermediate image layers).
       The default is false.

   --digests
       Show image digests

   --filter, -f=filter
       Provide filter values.

       The filters argument format is of key=value or key!=value. If there  is
       more  than  one  filter,  then  pass multiple OPTIONS: --filter foo=bar
       --filter bif=baz.

       Supported filters:

       ┌─────────────┬────────────────────────────┐
       │Filter       │ Description                │
       ├─────────────┼────────────────────────────┤
       │id           │ Filter by image id.        │
    

In [10]:
man podman-system-prune

podman-system-prune(1)      General Commands Manual     podman-system-prune(1)

NAME
       podman-system-prune  - Remove all unused pods, containers, images, net‐
       works, and volume data

SYNOPSIS
       podman system prune [options]

DESCRIPTION
       podman system prune removes all unused containers  (both  dangling  and
       unreferenced), pods, networks, and optionally, volumes from local stor‐
       age.

       With the --all option, you can delete all unused images.  Unused images
       are  dangling  images  as well as any image that does not have any con‐
       tainers based on it.

       By default, volumes are not removed to prevent important data from  be‐
       ing  deleted  if  there is currently no container using the volume. Use
       the --volumes flag when running the command to prune volumes as well.

OPTIONS
   --all, -a
       Recursively remove all unused pods, containers, images,  networks,  and
       volume data. (Maximum 50 iterations.)

   --f

## Inspecting images

### `podman image inspect`

Sometimes you may want to examine the configuration of an image; use the `podman image inspect` command for this. The `podman inspect` command can also be used to inspect images, but the names can conflict with containers, so I prefer to use the specific image command:

In [15]:
podman image inspect myimage | wc -l

489


```json
[
  {
    "Id": "3d733e82ab2b67409b2fa956eeedb5869e468a12bd93d98980ed6a2307d30965",
    "Digest": "sha256:57c753c4ebd70df789392672c7a9fa7ff6167ee3f925594a040390b319af8589",
    "RepoTags": [
      "localhost/myimage:latest"
    ],
    "RepoDigests": [
      "localhost/myimage@sha256:57c753c4ebd70df789392672c7a9fa7ff6167ee3f925594a040390b319af8589"
    ],
    "Parent": "203593be2e5cc693e0dcbffc46d601cf34c541c8fc0632dadfeee69ef6532e59",
    "Comment": "",
    "Created": "2024-01-24T14:43:00.869812063Z",
    "Config": {
      "User": "1001",
      "ExposedPorts": {
        "8080/tcp": {},
        "8443/tcp": {}
      },
      "Env": [
        "HTTPD_VAR_RUN=/var/run/httpd",
        "HTTPD_TLS_CERT_PATH=/etc/httpd/tls",
        "HTTPD_CONFIGURATION_PATH=/opt/app-root/etc/httpd.d",
        "HTTPD_MAIN_CONF_MODULES_D_PATH=/etc/httpd/conf.modules.d",
        "APP_ROOT=/opt/app-root",
        "HTTPD_LOG_PATH=/var/log/httpd",
        "HTTPD_MAIN_CONF_PATH=/etc/httpd/conf",
        "HTTPD_MAIN_CONF_D_PATH=/etc/httpd/conf.d",
        "HTTPD_DATA_PATH=/var/www",
        "HTTPD_CONTAINER_SCRIPTS_PATH=/usr/share/container-scripts/httpd/",
        "PATH=/opt/app-root/src/bin:/opt/app-root/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin",
        "TERM=xterm",
        "container=oci",
        "STI_SCRIPTS_URL=image:///usr/libexec/s2i",
        "DESCRIPTION=Apache httpd 2.4 available as container, is a powerful, efficient, and extensible web server. Apache supports a variety of features, many implemented as compiled modules which extend the core functionality. These can range from server-side programming language support to authentication schemes. Virtual hosting allows one Apache installation to serve many different Web sites.",
        "PLATFORM=el8",
        "HTTPD_VERSION=2.4",
        "HTTPD_DATA_ORIG_PATH=/var/www",
        "STI_SCRIPTS_PATH=/usr/libexec/s2i",
        "SUMMARY=Platform for running Apache httpd 2.4 or building httpd-based application",
        "HTTPD_APP_ROOT=/opt/app-root",
        "HOME=/opt/app-root/src"
      ],
      "Entrypoint": [
        "container-entrypoint"
      ],
      "Cmd": [
        "/usr/bin/run-httpd"
      ],
    ...
```

As you can see, this command outputs a large JSON array that includes the data used for the OCI Image Format specification. When you create a container from an image, this information is used as one of the inputs to create the container.

When using the `inspect` command, it is often better to pipe its output to `less` or `grep` to find particular fields you are interested in. Alternatively, you can use the `--format` option.

### `--format`

If you want to to examine the default command to be executed from this image, execute the following:

In [16]:
podman image inspect --format '{{ .Config.Cmd }}' myimage

[/usr/bin/run-httpd]


Or if you want to see the stop signal, execute:

In [17]:
podman image inspect --format '{{ .Config.StopSignal }}' myimage

As you can see, nothing is output, meaning the developer of the application did not specify a `STOPSIGNAL`. When you build a container off of this image, the `STOPSIGNAL` is the default, `15`, unless you override it via the command line.

One notable `podman image inspect` option is the following:

- `--format` — This is useful as you see above to extract particular fields out of the json.

In [19]:
man podman-image-inspect

podman-image-inspect(1)     General Commands Manual    podman-image-inspect(1)

NAME
       podman-image-inspect - Display an image's configuration

SYNOPSIS
       podman image inspect [options] image [image ...]

DESCRIPTION
       This displays the low-level information on images identified by name or
       ID. By default, this will render all results in a  JSON  array.   If  a
       format  is  specified, the given template will be executed for each re‐
       sult.

OPTIONS
   --format, -f=format
       Format the output using the given Go template.  The  keys  of  the  re‐
       turned  JSON can be used as the values for the --format flag (see exam‐
       ples below).

EXAMPLE
              $ podman image inspect fedora
              [
                  {
                      "Id": "37e5619f4a8ca9dbc4d6c0ae7890625674a10dbcfb76201399e2aaddb40da17d",
                      "Digest": "sha256:1b0d4ddd99b1a8c8a80e885aafe6034c95f266da44ead992aab388e6aa91611a",
                     

## Login to the registry

### `podman login`

To follow along in this section, you need to set up an account at a container registry; there are several container registries available to choose from. 

***
_ChatGPT:_  
- **Docker Hub**: Docker Hub offers a free tier that allows users to push public repositories for free. Private repositories require a paid subscription.

- **GitHub Container Registry**: GitHub offers a container registry as part of its services, allowing users to push and pull container images for free. Private repositories are also supported at no additional cost.

- **Google Container Registry**: Google Cloud Platform's Container Registry offers a free tier with a limited number of storage and data transfer, allowing users to push and pull container images.

- **Amazon Elastic Container Registry (ECR)**: AWS ECR offers a free tier for the first 12 months, allowing users to store, manage, and deploy Docker container images for free with certain limits.

- **Quay.io**: Quay.io offers a free tier that allows users to push and pull public repositories for free. Private repositories require a paid subscription.
***

For the examples, I will continue to use my `rhatdan` account at `quay.io`. Log in to get your credentials:

```sh
$ podman login quay.io
Username: rhatdan
Password: 
Login Succeeded!
```

Notice the Podman command prompts you for your username and password at the registry. The `podman login` command has options to pass the username/password information on the command line to avoid the prompt, allowing you to automate the login process:

In [1]:
podman login -u your_username --password-stdin < password_file.txt docker.io

Login Succeeded!


It is possible to run the `podman login` command multiple times to log in to multiple registries, storing the login information in the same authorization file with a different stanza.

> NOTE: Podman supports other mechanisms for storing the password information. These are called **credential helpers**.

#### `--password-stdin`

_ChatGPT_:  
The `--password-stdin` option in bash is used to read the password from standard input instead of prompting the user for it when running a command that requires a password. This option is commonly used with commands that require authentication, such as `docker login`, to securely provide the password without displaying it in the command line or script. It allows the password to be passed through standard input, which can be helpful for automation and scripting purposes. Using `--password-stdin` allows you to provide the password securely without it being exposed in the process list. The password is read directly from the input stream without being echoed in the terminal or visible in the process list.

Four different ways to pass the password:

- This can be particularly useful when scripting the login process for automation, but the password can potentially be visible in the process list or command history:
```sh
echo "your_password" | podman login -u your_username --password-stdin your_registry
```
- `printf` command along with input redirection provides the password to `podman login` in a more secure manner - without exposing it in the process list or command history:
```sh
printf '%s' 'your_password' | podman login -u your_username --password-stdin your_registry
```
- Use a password file to avoid exposing the password in the command history or process list:
```sh
podman login -u your_username  --password-stdin < password_file.txt your_registry
or
cat password_file.txt | podman login -u your_username --password-stdin your_registry
```

#### `auth.json`

To store authentication information for the user, the `podman login` command creates an `auth.json` file. By default, this is stored in the `/run/user/$UID/containers/auth.json` file:

```sh
cat /run/user/$UID/containers/auth.json
{
        "auths": {
                "docker.io": {
                        "auth": "OBSCURED-BASE64-PASSWORD"
                },
                "ghrc.io": {
                        "auth": "OBSCURED-BASE64-PASSWORD"
                }
        }
}
```

The `auth.json` file contains your registry password in a Base64-encoded string; there is no cryptography involved. Therefore, the `auth.json` file needs to be protected. Podman defaults to storing the file in `/run` because it is a temporary filesystem and is destroyed when you log out or the system is rebooted. The `/run/user/$UID/containers` directory is not accessible by other users on the system:

In [4]:
ls -l /run/user/$UID/containers/auth.json

-rw------- 1 commi commi 219 Feb  1 00:27 /run/user/1000/containers/auth.json


It is possible to override the location by specifying the `--authfile` option. Alternatively, you can use the `REGISTRY_AUTH_FILE` environment variable to modify its location. If both are specified, the `--authfile` option is used. All container tools use this file to access the container registry.

_ChatGPT:_  
If you define the `REGISTRY_AUTH_FILE` environment variable, Podman will use the path specified in this variable as the default location for storing the authorization information. Additionally, if you use the `--authfile` option while running the `podman login` command, it will take precedence over the `REGISTRY_AUTH_FILE` variable and use the specified auth file instead.

So, in summary, when `REGISTRY_AUTH_FILE` is defined, it acts as the default path for the auth file, but you can override it by specifying `--authfile` in the `podman login` command.

#### `--get-login`

In [7]:
podman login --get-login docker.io

lefthand67


In [8]:
podman login --get-login ghrc.io

lefthand67


### `podman logout`

After you are done using the registry, you can log out by executing `podman logout`. This command deletes the cached credentials stored in the `auth.json` file:

```
$ podman logout quay.io
Removed login credentials for quay.io
```

In [7]:
podman logout ghcr.io

Removed login credentials for ghcr.io


Some notable `podman login` and `logout` options include the following:

- `--username`, or `-u` — This provides the Podman username to use when logging into the registry.
- `--authfile` — This tells Podman to store the authorization file in a different location. You can also use the `REGISTRY_AUTH_FILE` environment variable to change the location.
- `--all` — This allows you to log out of all of the registries.

In [5]:
man podman-login

podman-login(1)             General Commands Manual            podman-login(1)

NAME
       podman-login - Login to a container registry

SYNOPSIS
       podman login [options] [registry]

DESCRIPTION
       podman  login  logs  into  a specified registry server with the correct
       username and password. If the registry is not specified, the first reg‐
       istry under [registries.search] from registries.conf will be used. pod‐
       man login reads in the username and password from STDIN.  The  username
       and  password  can  also  be set using the username and password flags.
       The path of the authentication file can be specified  by  the  user  by
       setting  the  authfile  flag.  The default path for reading and writing
       credentials is  ${XDG_RUNTIME_DIR}/containers/auth.json.   Podman  will
       use existing credentials if the user does not pass in a username.  Pod‐
       man will first search for the username and password in  the  ${XDG_RUN‐
       TI

In [6]:
man podman-logout

podman-logout(1)            General Commands Manual           podman-logout(1)

NAME
       podman-logout - Logout of a container registry

SYNOPSIS
       podman logout [options] registry

DESCRIPTION
       podman  logout  logs out of a specified registry server by deleting the
       cached credentials stored in the auth.json file. If the registry is not
       specified,  the  first  registry  under  [registries.search]  from reg‐
       istries.conf will be used. The path of the authentication file  can  be
       overridden  by the user by setting the authfile flag.  The default path
       used is ${XDG_RUNTIME_DIR}/containers/auth.json. For more details about
       format  and  configurations of the auth,json file, please refer to con‐
       tainers-auth.json(5) All the cached credentials can be removed by  set‐
       ting the all flag.

       podman [GLOBAL OPTIONS]

       podman logout [GLOBAL OPTIONS]

       podman logout [OPTIONS] REGISTRY [GLOBAL OPTIONS]

OPTIONS
  

In Podman, you use the `podman push` command to copy an image and all of its layers out of container storage and push it to other forms of container image storage, like a container registry. Podman supports a few different types of container storage, which it calls **transports**.

## Pushing images

### Container transports

> _ChatGPT_: **Container transports** are the methods or protocols used to fetch container images and distribute them. Container transports are essential for moving container images between different environments, systems, or container registries. Podman supports multiple container transports to provide flexibility and compatibility with various container image sources.

Podman uses the [`containers/image` library](https://github.com/containers/image) for pulling and pushing images. I describe the `containers/image` project as a library for copying images between different types of container storage. One storage, as you have seen, is `containers/storage`.

When pushing an image, the `[destination]` is specified using `transport:ImageName` format. If no transport is specified, the `docker` (container registry) transport is used by default.

One of the novel things that Docker did, as I explained earlier, was invent the container registry concept — basically, a web server that contains container images. The `docker.io`, `quay.io`, and `Artifactory` web servers are all examples of container registries. The Docker engineering team defined a protocol for pulling and pushing these images from the container registries, which I refer to as the **container registry** or **docker transport**.

When I want to run a container of an image, I can fully specify the image name, including the transport like the following command: 

In [20]:
podman run docker://registry.access.redhat.com/ubi8/httpd-24:latest echo hello

hello


For Podman, `docker://` transport is the default; it can be skipped for convenience:

In [21]:
podman run registry.access.redhat.com/ubi8/httpd-24:latest echo hello

hello


Table 2.3 describes the supported transports for different types of container’s storage.

<center>
    Table 2.3 Podman-supported transports<br>
<br>

|Transport|Description|
|-|-|
|Container registry (Docker)|Default transport. This references a container image stored in a remote container image registry. Container registry is a place for storing and sharing container images (e.g., docker.io or quay.io).
|oci|References a container image, compliant with the Open Container Image Layout Specification. The manifest and layer tarballs as individual files are located in the local directory.
|dir|References a container image, compliant with the Docker image layout. It is very similar to the oci transport but stores the files using the legacy Docker format. It is a nonstandardized format, primarily useful for debugging or noninvasive container inspection.
|docker-archive|References a container image in Docker image layout, which is packed into a TAR archive.
|oci-archive|References an image compliant with the Open Container Image Layout Specification, which is packed into a TAR archive. It is very similar to the docker-archive transport, but it stores an image in OCI format.
|docker-daemon|References an image stored in the Docker daemon’s internal storage. Since the Docker daemon requires root privileges, Podman has to be run by the root user.
|container-storage|References an image located in a local container storage. It is not a transport but more of a mechanism for storing images. It can be used to convert other transports into container-storage. |Podman defaults to using container-storage for local images.
    
> _ChatGPT_: When working with Podman, you often don't need to specify the container transport explicitly. Podman can infer the transport based on the format of the image reference. However, there are cases, especially when dealing with non-default transports or specific scenarios, where explicitly specifying the container transport can be beneficial.

### `podman push`

The `myimage` image you created in the previous section was created locally, which means it doesn’t have a registry associated with it. By default, locally created images have the localhost registry associated with them. You can see the images in the `containers/storage` using the `podman images` command:

In [22]:
podman images

REPOSITORY                                TAG         IMAGE ID      CREATED      SIZE
localhost/myimage                         latest      3d733e82ab2b  5 hours ago  459 MB
registry.access.redhat.com/ubi8/httpd-24  latest      203593be2e5c  3 weeks ago  458 MB


If the image has a remote registry associated with it (e.g., `registry.access.redhat.com/ubi8`), it can be pushed without specifying the `[destination]` field. On the contrary, since `localhost/myimage` does not have a registry associated with it, remote registry needs to be specified (e.g., `quay.io/rhatdan`):

```sh
$ podman push myimage quay.io/rhatdan/myimage
```
```
Getting image source signatures
Copying blob 164d51196137 done
Copying blob 8f26704f753c done 
Copying blob 83310c7c677c done 
Copying blob 654b3bf1361e [==================>-------------------] 82.0MiB / 162.7MiB
Copying blob e39c3abf0df9 [================>---------------------] 100.0MiB / 222.8MiB
```

> NOTE: Before executing the `podman push` command, I logged into the `quay.io/` `rhatdan` account using `podman login`.

After the `push` command is finished, the image becomes available for `pull` for other users, given they have access to this container registry. 

In [8]:
man podman-push

podman-push(1)              General Commands Manual             podman-push(1)

NAME
       podman-push  -  Push  an image, manifest list or image index from local
       storage to elsewhere

SYNOPSIS
       podman push [options] image [destination]

       podman image push [options] image [destination]

DESCRIPTION
       Pushes an image, manifest list or image index from local storage  to  a
       specified  destination.  Push  is  mainly  used  to push images to reg‐
       istries, however podman push can be used to save images to tarballs and
       directories  using  the  following  transports:  dir:, docker-archive:,
       docker-daemon: and oci-archive:.

Image storage
       Images are pushed from those stored in local image storage.

DESTINATION
       DESTINATION is the location the container image is pushed to.  It  sup‐
       ports  all transports from containers-transports(5). If no transport is
       specified, the docker (i.e., container  registry)  transport  is

## `podman tag` | Tagging images

Notice when you pushed the image to a container registry, you renamed `myimage` to `quay.io/rhatdan/myimage`. It’d be nice to just have the local image named `quay.io/rhatdan/myimage`, in which case you could have just executed

```sh
$ podman push quay.io/rhatdan/myimage
```

Earlier in this chapter, I pointed out that locally created images are created with a `localhost` registry. Images get created with the localhost registry when you `commit` a container to an image or if you use `podman build` to build an image. Podman has a mechanism to add additional names to images; it calls these **names tags**, and the command is `podman tag`.

Using the `podman images` command, list the image(s) in `container/storage`:

In [9]:
podman images

REPOSITORY               TAG         IMAGE ID      CREATED         SIZE
localhost/myimage        latest      17da1e51f929  48 minutes ago  173 MB
docker.io/library/httpd  latest      2cfd65f8d6ff  2 weeks ago     173 MB


You will want the final image you plan on shipping to be referred to as `docker.io/rhatdan/myimage`. To achieve this, add that name with the following podman tag command:

In [10]:
podman tag myimage docker.io/lefthand67/test1

Notice that the `localhost/myimage` and `quay.io/ rhatdan/myimage` have the same image ID:

In [11]:
podman images

REPOSITORY                  TAG         IMAGE ID      CREATED            SIZE
localhost/myimage           latest      17da1e51f929  About an hour ago  173 MB
docker.io/lefthand67/test1  latest      17da1e51f929  About an hour ago  173 MB
docker.io/library/httpd     latest      2cfd65f8d6ff  2 weeks ago        173 MB


Since the images have the same image ID, they are the same image with multiple names. Now you can interact directly with `quay.io/rhatdan/myimage` and push without requiring the destination name:

```sh
$ podman push quay.io/rhatdan/myimage
```
```
Getting image source signatures
...
Storing signatures
```

That was much simpler.

Let’s tag the previously used image with a version, 1.0:

```sh
$ podman tag quay.io/rhatdan/myimage quay.io/rhatdan/myimage:1.0
```

Once again, examine the images; notice that myimage now has three different names/tags. All three have the same image ID:

In [13]:
podman images

REPOSITORY                  TAG         IMAGE ID      CREATED      SIZE
localhost/myimage           latest      17da1e51f929  2 hours ago  173 MB
docker.io/lefthand67/test1  latest      17da1e51f929  2 hours ago  173 MB
docker.io/lefthand67/test1  1.0         17da1e51f929  2 hours ago  173 MB
docker.io/library/httpd     latest      2cfd65f8d6ff  2 weeks ago  173 MB


Now you can push the 1.0 version of the `myimage` (application) to the registry:

```sh
$ podman push quay.io/rhatdan/myimage:1.0
Getting image source signatures
Copying blob 8f26704f753c skipped: already exists 
Copying blob e39c3abf0df9 skipped: already exists 
Copying blob 654b3bf1361e skipped: already exists 
Copying blob 83310c7c677c skipped: already exists 
Copying blob 164d51196137 [--------------------------------------] 0.0b / 0.0b
Copying config 2c7e43d880 [--------------------------------------] 0.0b / 4.0KiB
Writing manifest to image destination
Storing signatures
```

Users can pull either the `latest` image or the `1.0` version. Later, when you build version `2.0` of your application, you can store both images at the registry. You can run both version `1.0` and `2.0` of your application on the host at the same time.

Use a web browser (e.g., Firefox, Chrome, Safari, Internet Explorer, or Microsoft Edge) to look at the images at `quay.io`. You can see `1.0` and the latest image in figure 2.4:

![](../data/images/Screenshot_20240201_014641.png)

> NOTE: Contrary to common sense, the tag `latest` does not refer to the most up-to-date image in the repository. It is just another tag with no magic involved. Even worse, because it is being used as a default tag for images pushed without tags, it could refer to any random version of an image. There could be newer images in the container registry than your local container’s storage with this tag. Thus, it is always **better to refer to the specific version of the image you want to use, rather than relying on the latest**.

In [18]:
man podman-tag

podman-tag(1)               General Commands Manual              podman-tag(1)

NAME
       podman-tag - Add an additional name to a local image

SYNOPSIS
       podman tag image[:tag] [target-name[:tag]...] [options]

       podman image tag image[:tag] [target-name[:tag]...] [options]

DESCRIPTION
       Assigns  a  new image name to an existing image.  A full name refers to
       the entire image name, including the optional  tag  after  the  :.   If
       there  is  no tag provided, then podman will default to latest for both
       the image and the target-name.

OPTIONS
   --help, -h
       Print usage statement

EXAMPLES
              $ podman tag 0e3bbc2 fedora:latest

              $ podman tag httpd myregistryhost:5000/fedora/httpd:v2

SEE ALSO
       podman(1)

HISTORY
       December 2019, Update description to refer to 'name' instead of 'alias'
       by  Sascha  Grunert  sgrunert@suse.com  ⟨mailto:sgrunert@suse.com⟩ July
       2017,   Originally   compiled   by    Ryan

## Removing images

### `podman rmi`

In [17]:
podman images

REPOSITORY                  TAG         IMAGE ID      CREATED      SIZE
localhost/myimage           latest      17da1e51f929  2 hours ago  173 MB
docker.io/lefthand67/test1  latest      17da1e51f929  2 hours ago  173 MB
docker.io/lefthand67/test1  1.0         17da1e51f929  2 hours ago  173 MB
docker.io/library/httpd     latest      2cfd65f8d6ff  2 weeks ago  173 MB


Use the `podman rmi` command to remove local images:

In [18]:
podman rmi localhost/myimage

Untagged: localhost/myimage:latest


In [19]:
podman images

REPOSITORY                  TAG         IMAGE ID      CREATED      SIZE
docker.io/lefthand67/test1  latest      17da1e51f929  2 hours ago  173 MB
docker.io/lefthand67/test1  1.0         17da1e51f929  2 hours ago  173 MB
docker.io/library/httpd     latest      2cfd65f8d6ff  2 weeks ago  173 MB


The command didn’t actually remove the image but only the localhost tag from the image. Podman still has two references to the same image `ID`: 

> the actual content of the image has not been removed. 

None of the disk space was freed up.

You can remove the other tags using a short name (see section 2.2.8). Podman uses the short name and finds the _first_ name in local storage that matches the short name without a registry and removes it, which is why I need to remove it twice to get rid of both images. Tags other than latest need to be specified explicitly:

```sh
$ podman rmi myimage
Untagged: quay.io/rhatdan/myimage:latest
$ podman rmi myimage:1.0
Untagged: quay.io/rhatdan/myimage:1.0
Deleted: 2c7e43d88038669e8cdbdff324a9f9605d99697215a0d21c360fe8dfa8471bab
```

It is only when the last tag is removed that the actual disk space is reclaimed.

Alternatively, you can try removing the images by specifying the image ID. But that fails because there are multiple tags for the same image.

In [24]:
podman images

REPOSITORY                  TAG         IMAGE ID      CREATED      SIZE
docker.io/lefthand67/test1  latest      17da1e51f929  2 hours ago  173 MB
docker.io/lefthand67/test1  1.0         17da1e51f929  2 hours ago  173 MB
docker.io/library/httpd     latest      2cfd65f8d6ff  2 weeks ago  173 MB


In [25]:
podman rmi 17da1e51f929

Error: unable to delete image "17da1e51f929bbfd8baa22a82e9ea1d5aad73b72de1541992bf107183dbc7a24" by ID with more than one tag ([docker.io/lefthand67/test1:latest docker.io/lefthand67/test1:1.0]): please force removal


: 125

Adding the `--force` option will remove the image and all of its tags. 

### `podman image prune`

As your image sizes and numbers grow and more containers are created, it becomes harder to figure out which images are no longer needed. Podman has another useful command — `podman image prune` — for removing all dangling images. 

> **Dangling images** are images that no longer have a tag associated with them or a container using them. 

The prune command also has the `--all` option, which removes all images that are currently not in use by any containers, including dangling images:

In [29]:
podman image prune -a

WARNING! This command removes all images without at least one container associated with them.
Are you sure you want to continue? [y/N] 


```
WARNING! This command removes all images without at least one container associated with them.
Are you sure you want to continue? [y/N] y
17da1e51f929bbfd8baa22a82e9ea1d5aad73b72de1541992bf107183dbc7a24
```

In [32]:
podman ps

CONTAINER ID  IMAGE                           COMMAND           CREATED      STATUS          PORTS                 NAMES
df5f8cc700dd  docker.io/library/httpd:latest  httpd-foreground  2 hours ago  Up 2 hours ago  0.0.0.0:8080->80/tcp  myapp


In [31]:
podman images

REPOSITORY               TAG         IMAGE ID      CREATED      SIZE
docker.io/library/httpd  latest      2cfd65f8d6ff  2 weeks ago  173 MB


Some notable `podman rmi` options include the following:

- `--all` — This tells Podman to remove all images, freeing up all storage. Images that have containers running on them are not removed.
- `--force` — This tells Podman to remove all containers that  are  using the image before removing the image from the system.

> NOTE: Having no containers running the `podman image prune` command removes all of the local images. This frees up all of the disk space in the home directory. You can use the `podman system df` command to show all of the storage in your home directory used by Podman.

In [41]:
podman ps -a

CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES


In [43]:
podman images

REPOSITORY  TAG         IMAGE ID    CREATED     SIZE


### `podman system df`

In [30]:
podman system df

TYPE           TOTAL       ACTIVE      SIZE        RECLAIMABLE
Images         1           1           172.7MB     0B (0%)
Containers     1           1           99B         0B (0%)
Local Volumes  0           0           0B          0B (0%)


In [34]:
man podman rmi

podman-rmi(1)               General Commands Manual              podman-rmi(1)

NAME
       podman-rmi - Removes one or more locally stored images

SYNOPSIS
       podman rmi [options] image [...]

       podman image rm [options] image [...]

DESCRIPTION
       Removes  one  or more locally stored images.  Passing an argument image
       deletes it, along with any of its dangling parent images.   A  dangling
       image is an image without a tag and without being referenced by another
       image.

       Note: To delete an image from a remote registry, use the skopeo  delete
       command.  Some  registries  do not allow users to delete an image via a
       CLI remotely.

OPTIONS
   --all, -a
       Remove all images in the local storage.

   --force, -f
       This option will cause podman to remove all containers that  are  using
       the image before removing the image from the system.

   --ignore, -i
       If a specified image does not exist in the local storage, ignore 

In [39]:
man podman-image-prune

podman-image-prune(1)       General Commands Manual      podman-image-prune(1)

NAME
       podman-image-prune - Remove all unused images from the local store

SYNOPSIS
       podman image prune [options]

DESCRIPTION
       podman image prune removes all dangling images from local storage. With
       the all option, you can delete all unused images (i.e., images  not  in
       use by any container).

       The  image prune command does not prune cache images that only use lay‐
       ers that are necessary for other images.

OPTIONS
   --all, -a
       Remove dangling images and images that have no associated containers.

   --external
       Remove images even when they are used  by  external  containers  (e.g.,
       build containers).

   --filter=filters
       Provide filter values.

       The  filters argument format is of key=value. If there is more than one
       filter, then pass multiple OPTIONS: --filter foo=bar --filter bif=baz.

       Supported filters:

       ┌──

In [38]:
man podman-system-df

podman-system-df(1)         General Commands Manual        podman-system-df(1)

NAME
       podman-system-df - Show podman disk usage

SYNOPSIS
       podman system df [options]

DESCRIPTION
       Show podman disk usage

OPTIONS
   --format=format
       Pretty-print images using a Go template

   --verbose, -v
       Show detailed information on space usage

EXAMPLE
              $ podman system df
              TYPE            TOTAL   ACTIVE   SIZE    RECLAIMABLE
              Images          6       2        281MB   168MB (59%)
              Containers      3       1        0B      0B (0%)
              Local Volumes   1       1        22B     0B (0%)

              $ podman system df -v
              Images space usage:

              REPOSITORY                 TAG      IMAGE ID       CREATED       SIZE     SHARED SIZE   UNIQUE SIZE   CONTAINERS
              docker.io/library/alpine   latest   5cb3aa00f899   2 weeks ago   5.79MB   0B            5.79MB       5

              Conta

## `podman pull` | Pulling images

Although you previously removed all local images, you can pull the previously pushed image at `quay.io/rhatdan/myimage`. Podman has the podman pull command to pull images from container registries (transports) into local container storage:

```
$ podman pull docker.io/lefthand67/test1
Trying to pull docker.io/lefthand67/test1:latest...
Getting image source signatures
Copying blob 376771e8483c done  
Copying blob fd0579f22872 done  
Copying blob 6a6627aecff0 done  
Copying blob 4f4fb700ef54 done  
Copying blob 2f44b7a888fa done  
Copying blob 152f4888b550 done  
Copying blob 822fa9f2f07b done  
Copying config 17da1e51f9 done  
Writing manifest to image destination
Storing signatures
17da1e51f929bbfd8baa22a82e9ea1d5aad73b72de1541992bf107183dbc7a24
```

You probably remember similar output from the `podman run` command from section 2.1.2:

```sh
$ podman run -d -p 8081:8080 --name myapp1 registry.access.redhat.com/ubi8/httpd-24
```

Many Podman commands implicitly execute the `podman pull` command if the required image is not present locally.

So executing `podman images` shows the image back in container storage, ready to be used for containers:

In [2]:
podman images

REPOSITORY                  TAG         IMAGE ID      CREATED     SIZE
docker.io/lefthand67/test1  latest      17da1e51f929  2 days ago  173 MB


Up until now, you have been typing the image with the full names as `registry.access .redhat.com/ubi8/httpd-24 or quay.io/rhatdan/myimage`, but if you are like me and not a great typist, this can be a pain. You really need a way to refer to the images via short names.

### Short names and container registries

When Docker first hit the scene, they defined an **image reference** as a combination of the 
- container registry where the image was stored, 
- repository, 
- image name, and 
- a tag or version of the image. 

In our examples, we have been using `quay.io/rhatdan/myimage`. In table 2.4, you can see this image name breakdown; note that the `latest` tag was used implicitly, as the image version wasn’t specified.

<center>
Table 2.4 Container image name table</center>

|Registry|Repository|Name|Tag|
|-|-|-|-|
|quay.io|rhatdan|myimage|latest|
    
The Docker command line has internally set the `docker.io` registry as the only registry, thus making every short image name refer to images at `docker.io`. There is also a special repository library, which is used for certified images.

So rather than typing:

```
# docker pull docker.io/library/alpine:latest
```

You can just execute

```
# docker pull alpine
```

Conversely, if you want to pull an image from a different registry, you need to specify the full name of the image:

```
# docker pull registry.access.redhat.com/ubi8/httpd-24:latest
```

Table 2.5 shows the difference between the image name used in a short name versus the fully specified image name. Notice that when using the short name, the registry, repository, and tag were not specified.

<center>
Table 2.5 Short name to container image name table</center>

|Registry|Repository|Name|Tag|
|-|-|-|-|
|||alpine||
|docker.i|librar|alpin|lates|

Since I am lazy and hate to type extra characters, I almost always use short names. With Podman, the developers did not want to hardcode one registry, `docker.io`, into the tool.

### `podman info`

Podman allows distributions, companies, and you to control which registries to use and to be able to configure multiple registries. At the same time, Podman provides support for the easier-to-use short names.

Podman usually comes with multiple registries defined, controlled by the distribution that packaged Podman. You can use the `podman info` command to see what registries are defined for your Podman installation:

In [8]:
podman info -f json | jq ".registries"

{
  "search": [
    "docker.io",
    "ghrc.io",
    "quay.io"
  ]
}


The list of registries can be modified in the `registries.conf` file, which is described in section 5.2.1.

In [11]:
cat $HOME/.config/containers/registries.conf

unqualified-search-registries = ['docker.io', 'ghrc.io', 'quay.io']


In [6]:
man podman-info

podman-info(1)              General Commands Manual             podman-info(1)

NAME
       podman-info - Displays Podman related system information

SYNOPSIS
       podman info [options]

       podman system info [options]

DESCRIPTION
       Displays information pertinent to the host, current storage stats, con‐
       figured container registries, and build of podman.

OPTIONS
   --format, -f=format
       Change output format to "json" or a Go template.

       ┌────────────────┬────────────────────────────┐
       │Placeholder     │ Info pertaining to ...     │
       ├────────────────┼────────────────────────────┤
       │.Host ...       │ ...the host on which  pod‐ │
       │                │ man is running             │
       ├────────────────┼────────────────────────────┤
       │.Plugins ...    │ ...external plugins        │
       ├────────────────┼────────────────────────────┤
       │.Registries ... │ ...configured registries   │
       ├────────────────┼────────────────

### Security perspective

Let’s discuss the security side of things using these commands:

```sh
$ podman pull rhatdan/myimage
$ podman pull quay.io/rhatdan/myimage
```

From a security perspective, 

> it is always better to specify the full image name when pulling it from a registry. 

That way, Podman guarantees that it pulls from the specified registry. Imagine you are attempting to pull `rhatdan/myimage`. Using the previous search order, there is a chance someone could set up an account on `docker.io/rhatdan` and trick you into mistakenly pulling `docker.io/rhatdan/myimage`.

To help protect against this, on the first pull of an image, Podman prompts you to select an exact image from the list of found images in configured registries:

```sh
$ podman create -p 8080:8080 ubi8/httpd-24
```
```
? Please select an image: 
   registry.fedoraproject.org/ubi8/httpd-24:latest
 ▸ registry.access.redhat.com/ubi8/httpd-24:latest
   docker.io/ubi8/httpd-24:latest
   quay.io/ubi8/httpd-24:latest
```

Once you have selected and pulled an image successfully, Podman records the short name mapping. In the future, when you run a container with this short name, Podman uses the short name mapping to pick the correct registry and does not prompt.

Linux distributions also ship mappings of the most commonly used short names, as they want you to pull from their supported registries. You can find these short name configuration files in the `/etc/containers/registries.conf.d` directory on the Linux host. Companies can also drop short name alias files in this directory:

```sh
$ cat /etc/containers/registries.conf.d/000-shortnames.conf
```
```
[aliases]
  # centos
  "centos" = "quay.io/centos/centos"
  # containers
  "skopeo" = "quay.io/skopeo/stable"
  "buildah" = "quay.io/buildah/stable"
  "podman" = "quay.io/podman/stable"
...
```

In [16]:
cat /etc/containers/registries.conf.d/shortnames.conf

[aliases]
  # almalinux
  "almalinux" = "docker.io/library/almalinux"
  "almalinux-minimal" = "docker.io/library/almalinux-minimal"
  # centos
  "centos" = "quay.io/centos/centos"
  # containers
  "skopeo" = "quay.io/skopeo/stable"
  "buildah" = "quay.io/buildah/stable"
  "podman" = "quay.io/podman/stable"
  # docker
  "alpine" = "docker.io/library/alpine"
  "docker" = "docker.io/library/docker"
  "registry" = "docker.io/library/registry"
  "hello-world" = "docker.io/library/hello-world"
  "swarm" = "docker.io/library/swarm"
  # Fedora
  "fedora-minimal" = "registry.fedoraproject.org/fedora-minimal"
  "fedora" = "registry.fedoraproject.org/fedora"
  # openSUSE
  "opensuse/tumbleweed" = "registry.opensuse.org/opensuse/tumbleweed"
  "opensuse/tumbleweed-dnf" = "registry.opensuse.org/opensuse/tumbleweed-dnf"
  "opensuse/tumbleweed-microdnf" = "registry.opensuse.org/opensuse/tumbleweed-microdnf"
  "opensuse/leap" = "registry.opensuse.org/opensuse/leap"
  "opensuse/busybox" = "registry.open

Some notable podman pull options include the following:

- `--arch` — This tells Podman to pull an image for a different architecture. For example, on my x86_64 machine, I can pull an arm64 image. By default, `podman pull` pulls images for the native architecture.
- `--quiet (-q)` — This tells Podman not to print out all the progress information. It just prints the image ID when it completes.

In [17]:
man podman-pull

podman-pull(1)              General Commands Manual             podman-pull(1)

NAME
       podman-pull - Pull an image from a registry

SYNOPSIS
       podman pull [options] source [source...]

       podman image pull [options] source [source...]

       podman pull [options] [transport]name[:tag|@digest]

       podman image pull [options] [transport]name[:tag|@digest]

DESCRIPTION
       podman pull copies an image from a registry onto the local machine. The
       command can pull one or more images.  If the  image  reference  in  the
       command line argument does not contain a registry, it is referred to as
       ashort-name reference. If the image is a 'short-name' reference, Podman
       will  prompt  the  user for the specific container registry to pull the
       image from, if an alias for the short-name has not  been  specified  in
       the  short-name-aliases.conf.  If an image tag is not specified, podman
       pull defaults to the image with the latest tag (if i

## `podman search` | Searching for images

I have mentioned a few images in this book, but there are thousands and thousands of images available. You need a mechanism to be able to search through these images for the perfect match.

You might not know the name of a particular image you want to run or use as a base for your own image. Podman provides the command `podman search`, which allows you to search container registries for matching names:

In [20]:
podman search docker.io/httpd

NAME                                      DESCRIPTION
docker.io/library/httpd                   The Apache HTTP Server Project
docker.io/clearlinux/httpd                httpd HyperText Transfer Protocol (HTTP) ser...
docker.io/paketobuildpacks/httpd          
docker.io/vulhub/httpd                    
docker.io/jitesoft/httpd                  Apache httpd on Alpine linux.
docker.io/openquantumsafe/httpd           Demo of post-quantum cryptography in Apache...
docker.io/wodby/httpd                     
docker.io/dockette/httpdump               
docker.io/betterweb/httpd                 
docker.io/dockette/apache                 Apache / HTTPD
docker.io/centos/httpd-24-centos7         Platform for running Apache httpd 2.4 or bui...
docker.io/manageiq/httpd                  Container with httpd, built on CentOS for Ma...
docker.io/centos/httpd-24-centos8         
docker.io/dockerpinata/httpd              
docker.io/19022021/httpd-connection_test  This httpd image will test the connectivit

In this example, we are searching for images that include the string `httpd` in their name on the repository `docker.io`.

Some notable podman search options include the following:

- `--no-trunc` — This tells Podman to show the full description of the image.
- `--format` — This allows you to customize which fields are displayed by Podman.

In [22]:
man podman-search

podman-search(1)            General Commands Manual           podman-search(1)

NAME
       podman-search - Search a registry for an image

SYNOPSIS
       podman search [options] term

DESCRIPTION
       podman  search searches a registry or a list of registries for a match‐
       ing image.  The user can specify which registry to search by  prefixing
       the  registry  in the search term (e.g., registry.fedoraproject.org/fe‐
       dora).  By default, all unqualified-search  registries  in  containers-
       registries.conf(5) are used.

       The  default number of results is 25. The number of results can be lim‐
       ited using the --limit flag.   If  more  than  one  registry  is  being
       searched, the limit will be applied to each registry. The output can be
       filtered using the --filter flag. To get all available images in a reg‐
       istry  without a specific search term, the user can just enter the reg‐
       istry name with a trailing "/" (example registr

## Mounting images

Up until now, you have seen several ways of managing and manipulating container images, including inspecting, pushing, pulling, and searching for them. But you have only been able to look at the contents of an image by running it as a container. One way to simplify the process is mounting a container image.

Often you might want to examine the contents of a container image, and one way to do this is launching a shell inside a running container from the image. The problem with this is that the tools you use to examine the container image might not be available within the container. There is also a security risk that the application in the container is malicious, making use of this container undesirable.

To help with these problems, Podman provides the `podman image mount` command to mount an image’s root filesystem in a read-only mode without creating a container from it. The mounted image becomes immediately available on the host system, allowing you to examine its contents.

Now try mounting the image you pulled previously:

In [25]:
podman mount docker.io/lefthand67/test1:1.0

Error: cannot run command "podman mount" in rootless mode, must execute `podman unshare` first


: 125

The reason for this error is that rootless mode does not allow mounting images. You need to enter a 
- user namespace and 
- separate mount namespace. 

### `podman unshare`

**Chapter 5** explains how most rootless Podman commands enter the user namespace and mount namespace when they execute. For now, it is enough to know that the `podman unshare` command enters the user and mount namespaces and will shut down when you execute the exit command of your shell.

> NOTE: The name **unshare** comes from the Linux syscall `unshare` (`man 2 unshare`). Linux also includes an unshare tool (`man 1 unshare`), which allows you to create namespaces by hand. Another low-level tool called `nsenter`, or **namespace enter** (`man 1 nsenter`), allows you to join processes to different namespaces. `Podman unshare` uses the same kernel features. It simplifies the process of creating and configuring namespaces and inserting processes into the namespaces.

The `podman unshare` command leaves you at a `#` prompt, where you can actually mount an image:

```sh
$ podman unshare
#
```

Mount the image, and save the location of the mounted filesystem in an environment variable:

```sh
mnt=$(podman image mount docker.io/lefthand67/test1:1.0)
```

Now you can actually examine the content of the image. Let’s print the contents of a file on the terminal:

```
# mnt=$(podman image mount docker.io/lefthand67/test1:1.0)
# cat $mnt/usr/local/apache2/htdocs/index.html
<html>
 <head>
 </head>
 <body>              
  <h1>Rot Front!</h1>
 </body>
</html>
```

When you are done, unmount the image, and exit the unshare session:

```
# podman image unmount quay.io/rhatdan/myimage
17da1e51f929bbfd8baa22a82e9ea1d5aad73b72de1541992bf107183dbc7a24
# exit
```

> NOTE: You have examined about a half of the `podman image` subcommands, arguably the most used ones. Refer to the `Podman man` pages for a full explanation of these and other subcommands of the `podman image` command.

In [23]:
man 2 unshare

unshare(2)                    System Calls Manual                   unshare(2)

NAME
       unshare - disassociate parts of the process execution context

LIBRARY
       Standard C library (libc, -lc)

SYNOPSIS
       #define _GNU_SOURCE
       #include <sched.h>

       int unshare(int flags);

DESCRIPTION
       unshare() allows a process (or thread) to disassociate parts of its ex‐
       ecution context that are currently being shared  with  other  processes
       (or  threads).   Part of the execution context, such as the mount name‐
       space, is shared implicitly when a new process is created using fork(2)
       or  vfork(2),  while other parts, such as virtual memory, may be shared
       by explicit request when creating a process or thread using clone(2).

       The main use of unshare() is to allow a process to control  its  shared
       execution context without creating a new process.

       The flags argument is a bit mask that specifies which parts of the exe‐
  

In [24]:
man 1 unshare

UNSHARE(1)                       User Commands                      UNSHARE(1)

NAME
       unshare - run program in new namespaces

SYNOPSIS
       unshare [options] [program [arguments]]

DESCRIPTION
       The unshare command creates new namespaces (as specified by the
       command-line options described below) and then executes the specified
       program. If program is not given, then "${SHELL}" is run (default:
       /bin/sh).

       By default, a new namespace persists only as long as it has member
       processes. A new namespace can be made persistent even when it has no
       member processes by bind mounting /proc/pid/ns/type files to a
       filesystem path. A namespace that has been made persistent in this way
       can subsequently be entered with nsenter(1) even after the program
       terminates (except PID namespaces where a permanently running init
       process is required). Once a persistent namespace is no longer needed,
       it can be unpersisted by u

In [26]:
man 1 nsenter

NSENTER(1)                       User Commands                      NSENTER(1)

NAME
       nsenter - run program in different namespaces

SYNOPSIS
       nsenter [options] [program [arguments]]

DESCRIPTION
       The nsenter command executes program in the namespace(s) that are
       specified in the command-line options (described below). If program is
       not given, then "${SHELL}" is run (default: /bin/sh).

       Enterable namespaces are:

       mount namespace
           Mounting and unmounting filesystems will not affect the rest of the
           system, except for filesystems which are explicitly marked as
           shared (with mount --make-shared; see /proc/self/mountinfo for the
           shared flag). For further details, see mount_namespaces(7) and the
           discussion of the CLONE_NEWNS flag in clone(2).

       UTS namespace
           Setting hostname or domainname will not affect the rest of the
           system. For further details, see uts_namespaces(

# 2.3 Building images

Now that you have a better understanding of containers and images, the next important step is updating your image. The main reasons for this are the need to update your application and the availability of new versions for the base image you use. You can build scripts to manually run the commands to build the image, but luckily, Podman optimized the experience.

So far you have been working with images, which were already created and uploaded to a container registry. The process of creating a container image is called **building**.

When building container images, you manage not only your application but also the image content used by this application. In the days prior to containers, you shipped applications as an RPM or DEB package, and then it was up to the distribution to make sure the other parts of the OS were kept up to date and secure. But in the container world, the container image includes the application along with a subset of the OS. 

> It is the developers’ responsibility to keep all of the image contents up to date and secure.

A coworker of mine, **Scott McCarty** (smccarty@redhat.com, [@fatherlinux](../containers_nomenclature.ipynb)), has a saying, “_Container images don’t age like wine but more like cheese. As the image gets older it gets stinky_.”

This means that if the developer doesn’t keep up with the security updates, the number of vulnerabilities in the image will grow at an alarming rate. Luckily for developers, Podman has a special mechanism for helping you with image building for your applications. The `podman build` command uses the [Buildah tool](https://github.com/containers/buildah) as a library to build container images; Buildah is covered in **appendix A**.

## `podman build`

The `podman build` uses a special text document called `Containerfile` or `Dockerfile` to automate the building of container images. This document lists commands used to build a container image.

> NOTE: The concept of a `Dockerfile` and its syntax was originally created for the Docker tool, developed by Docker, Inc. Podman defaults to using `Containerfile` for the name, which uses the exact same syntax. Dockerfile is supported as well for legacy purposes. The Docker build command does not support Containerfile by default but can use the Containerfile. You can specify the `-f` option: `# docker build -f Containerfile`.

## Containerfile, or Dockerfile

Containerfiles take many directives. I break these down into two categories, adding content to the container image and describing and documenting how to use the image.

### Adding content to an image

Recall back in section 1.1.2 that [I described](#Container-image-format) a container image as a directory on disk that looks like root on a Linux system. This directory is called a **rootfs**. Several of the directives in a container job are adding content to this `rootfs`. This `rootfs` eventually contains all of the content used to create your container image.

1. Every Containerfile must include a `FROM` line. 

> The `FROM` line specifies the image that the new image is based off, often called a **base image**. 

The `podman build` command supports a special image named `scratch`, which means to start your image with no content. When Podman sees the `FROM scratch` directive, it just allocates space in containers/storage for an empty `rootfs`, then `COPY` can be used to populate the `rootfs`. More often, the `FROM` directive uses an existing image. 

For example, `FROM registry.access.redhat.com/ubi8` causes Podman to pull the `ubi8` image from the `registry.access.redhat.com` container registry and copy it to container storage. `podman build` pulls the same image as the `podman pull` command you learned about in [section 2.2.8](#podman-pull-|-Pulling-images). When the image is pulled, Podman uses container storage to mount the image on the `rootfs` directory, using a copy on the write filesystem, like OverlayFS, where the other directives can start to add content. This image becomes the **base layer** of the `rootfs`.

2. The `COPY` directive is often used to copy files, directories, or tarballs off of the local host into the newly created `rootfs`. 

3. The `RUN` directive is one of the most commonly used Containerfile directives. 

`RUN` tells Podman to actually run a container on the image. Package management tools, like DNF/YUM and apt-get, are run to install packages from distributions onto your new image. The `RUN` directive runs any command within the container image as a container. The `podman build` command runs the commands with the same security constraints as the `podman run` command.

As an example, imagine you want to add the `ps` command to a container image; you can create a directive like the following. The `RUN` command executes a container, which updates all of the packages from the base image, and then installs the `procps-ns` package, which includes the `ps` command. Finally the containerized command executes `yum` to clean up after itself, so `cruft` is removed from the container image:

```
RUN yum -y update; yum -y install procps-ng; yum -y clean all
```

### Documenting how to use the image

Adding content to the container image is only half of what you need to do when creating a container image. You also need to describe and document how the image will be used when other users download and run your image.

Recall that back in section 1.1.2, [I also described](#Container-image-format) the JSON file that included the image specification. This specification describes 
- how the container image is to be run, the command, 
- which user to run it with, 

and other requirements of the image. The Containerfile also supports many directives, which tells Podman how to run containers. These include the following:

- The `ENTRYPOINT` and `CMD` directives — These instrument the image with the default command to be executed when users execute the image with `Podman run`. 
    - `CMD` is the actual command to run. 
    - `ENTRYPOINT` can cause the entire image to execute as a single command.<br>
</br>
- The `ENV` directive — This sets up the default environment variables to run when Podman runs a container on the image.

- The `EXPOSE` directive — This records the network ports for Podman to expose in containers based on the image. If you execute `podman run --publish-all ...`, Podman looks inside of the image for the `EXPOSE` network ports and connects them to the host.

<center>
<b>Table 2.6 Containerfile directives that update the image</b>
</center>

|Directive examples|Explanation|
|-|-|
|`FROM quay.io/rhatdan/myimage`|Sets the base image for subsequent instructions. Containerfiles must have FROM as their first instruction. The FROM may appear multiple times within a single Containerfile to create multiple build stages.|
|`ADD start.sh /usr/bin/start.sh`|Copies new files, directories, or remote file URLs to the filesystem of the container at a specified path.|
|`COPY start.sh /usr/bin/start.sh`|Copies files to the filesystem of the container at a specified path.|
|`RUN dnf -y update`|Executes commands in a new layer on top of the current image and commits the results. The committed image is used for the next step in the Containerfile.|
|`VOLUME /var/lib/mydata`|Creates a mount point with the specified name and marks it as holding externally mounted volumes from the native host or from other containers. For more on volumes, see chapter 3.|

Table 2.7 explains the directives used in a Containerfile to populate the OCI Runtime Specifications with information that tells container engines like Podman information about the image and how to run the image. 

<center>
<b>Table 2.7 Containerfile directives that define the OCI Runtime Specification</b>
</center>

Directive examples|Explanation
-|-
`CMD /usr/bin/start.sh`|Specifies the default command to run when launching a container off this image. If `CMD` is not specified, the parent image’s `CMD` is inherited. Note that `RUN` and `CMD` are very different. `RUN` runs the commands during the build process, while `CMD` is only used when a user launches the image without specifying a command.
`ENTRYPOINT “/bin/sh -c”`|Allows you to configure a container to run as an executable. The `ENTRYPOINT` instruction is not overwritten when arguments are passed to `podman run`. This allows arguments to be passed to the entrypoint — for instance, `podman run <image> -d` passes the `-d` argument to the `ENTRYPOINT`.
`ENV foo=”bar”`|Adds an environment variable to be used during both the image build and container execution.
`EXPOSE 8080`|Announces the port that containerized applications will be exposing. This does not actually map or open any ports.
`LABEL Description=”Web browser which displays Hello World”`|Adds metadata to an image.
`MAINTAINER Daniel Walsh`|Sets the `Author` field for the generated images.
`STOPSIGNAL SIGTERM`|Sets the default stop signal sent to the container to exit. The signal can be a valid unsigned number or a signal name in the format `SIGNAME`.
`USER apache`|Sets the user name (or UID) and group name (or GID) to use for any `RUN`, `CMD`, and `ENTRYPOINT` specified after it.
`ONBUILD`|Adds a trigger instruction to the image to be executed at a later time, when the image is used as the base for another build.
`WORKDIR /var/www/html`|Sets the working directory for `RUN`, `CMD`, `ENTRYPOINT`, and `COPY` directives. A directory will be created if it doesn’t exist.

In [1]:
man 5 containerfile

CONTAINERFILE(5)            Container User Manuals            CONTAINERFILE(5)

NAME
       Containerfile(Dockerfile)  - automate the steps of creating a container
       image

INTRODUCTION
       The Containerfile is a configuration file that automates the  steps  of
       creating  a container image. It is similar to a Makefile. Container en‐
       gines (Podman, Buildah, Docker) read instructions from  the  Container‐
       file  to  automate  the steps otherwise performed manually to create an
       image. To build an image, create a file called Containerfile.

       The Containerfile describes the steps taken to assemble the image. When
       the Containerfile has been created, call the buildah bud, podman build,
       docker build command, using the path of context directory that contains
       Containerfile as the argument. Podman and Buildah default to Container‐
       file and will fall back to Dockerfile.  Docker  only  will  search  for
       Dockerfile in the con

### Committing the image

When `podman build` finishes processing the Containerfile, it commits the image, using the same code as `podman commit` you learned about in [section 2.1.9](#podman-commit-|-Create-your-own-image). Basically, Podman TARs up all of the differences between the new content in the `rootfs` and the base image, pulled down by the `FROM` directive. Podman also commits the JSON file and saves this as an image in container storage. 

Now you can take the steps used to build out containerized applications and automate them using a Containerfile and `podman build`.

> TIP: Use the `--tag` option to name the new image you are creating with `podman build`. This tells Podman to add the specified tag or name to the image in container storage in the same way as the [`podman tag` command](#podman-tag-|-Tagging-images).

## Automating the building of our application

### Context directory

First, create a directory to put your Containerfile and any other content for the container image in. The directory is called a **context directory**:

In [31]:
mkdir -p data/test

Next, create the `index.html` file you plan to use in the containerized application in the `test` directory:

In [32]:
cat > data/test/index.html << _EOF
<html>
 <head>
 </head>
 <body>
 <h1>Reih dich ein in die Arbeiter Einheitsfront!</h1>
 </body>
</html>
_EOF

### Create a Containerfile

Next, create a simple `Containerfile` to build your application in the `test` directory. 

- The first line of the Containerfile is the `FROM` directive to pull the `httpd-24` image you are treating as your base image. 
- Then add a `COPY` command to copy the `index.html` file into the image. The `COPY` directive tells Podman to copy the `index.html` file out of the context directory (./data/test) and copy it to the `/usr/local/apache2/htdocs/index.html` file within the image:

In [33]:
cat > data/test/Containerfile << _EOF
FROM docker.io/library/httpd
COPY index.html /usr/local/apache2/htdocs/index.html
_EOF

In [34]:
cat data/test/Containerfile

FROM docker.io/library/httpd
COPY index.html /usr/local/apache2/htdocs/index.html


### Build the container

Finally, use `podman build` to build your containerized application. Specify the `--tag` (`-t`) to name the image `docker.io/username/test1`. You also need to specify the context directory `./data/test`:

```sh
$ podman build -t docker.io/username/test1 ./data/test
```
```
STEP 1/2: FROM docker.io/library/httpd
STEP 2/2: COPY index.html /usr/local/apache2/htdocs/index.html
COMMIT docker.io/username/test1
--> f46a288f141
Successfully tagged docker.io/username/test1:latest
f46a288f141ed7a28d7f94db255e16c0c308f0a3fa3f432ea6312ac809ef8e62
```

When the `podman build` command completes, it commits the image and tags (`-t`) it with the `docker.io/username/test1` name. It is now ready to be pushed to the container registry using the `podman push` command.

![](../data/images/Screenshot_20240206_021000.png)

In [39]:
podman images

REPOSITORY                  TAG         IMAGE ID      CREATED         SIZE
docker.io/lefthand67/test1  latest      b7e73f4c4bfb  11 minutes ago  172 MB
<none>                      <none>      4a0bc3b603d8  22 minutes ago  172 MB
<none>                      <none>      17da1e51f929  4 days ago      173 MB
docker.io/library/httpd     latest      59bcd61b45fd  2 weeks ago     172 MB


In [36]:
podman ps

CONTAINER ID  IMAGE                              COMMAND           CREATED        STATUS            PORTS                 NAMES
e2c605997ff0  docker.io/lefthand67/test1:latest  httpd-foreground  2 minutes ago  Up 2 minutes ago  0.0.0.0:8080->80/tcp  myapp


### Troubleshooting - VR

If the text appearance has not changed when you go to `localhost://8080`, the do these steps:

- `podman stop myapp`
- `podman rm myapp`
- `podman run -d -p 8080:80 --name myapp docker.io/username/test1` which is an updated image.

Also you can use `--no-cache` flag when building the image, but it did not help me.

### Automate process

Now you can set up a CI/CD system or even a simple `cron` job to regularly build and replace `testapplication`:

```sh
$ cat > test/automate.sh << _EOF
#!/bin/bash
podman build -t docker.io/username/test1 ./data/test
podman push docker.io/username/test1
_EOF
$ chmod 777 myapp/automate.sh
```

_ChatGPT:_  

automatically increment the version number (I have not tested it yet):
```sh
#!/bin/bash

# Define the version file
version_file="./data/version.txt"

# Read the current version from the file
current_version=$(cat "$version_file" 2>/dev/null)

# If the version file doesn't exist or is empty, start from 1
if [ -z "$current_version" ]; then
    current_version=0
fi

# Increment the version
new_version=$((current_version + 1))

# Update the version file with the new version
echo "$new_version" > "$version_file"

# Build the image with the incremented version
podman build -t docker.io/username/test1:$new_version ./data/test

# Push the image to the registry
podman push docker.io/username/test1:$new_version
```

### Test before push

Add some test scripts as well to make sure your application works the way it was designed before replacing the previous version. Let’s take a look at the images that were built:

In [40]:
podman images

REPOSITORY                  TAG         IMAGE ID      CREATED         SIZE
docker.io/lefthand67/test1  latest      b7e73f4c4bfb  11 minutes ago  172 MB
<none>                      <none>      4a0bc3b603d8  22 minutes ago  172 MB
<none>                      <none>      17da1e51f929  4 days ago      173 MB
docker.io/library/httpd     latest      59bcd61b45fd  2 weeks ago     172 MB


```sh
$ podman images
REPOSITORY                         TAG        IMAGE ID        CREATED    SIZE
quay.io/rhatdan/myimage         latest    f81b8ace4f13  2 minutes ago  462 MB
<none>                          <none>    2c7e43d88038     2 days ago  462 MB
registry.access.redhat
➥.com/ubi8/httpd-24            latest    8594be0a0b57    5 weeks ago  462 MB
```

Notice the old version of `quay.io/rhatdan/myimage`, image ID `2c7e43d88038`, still exists in container storage but now has a REPOSITORY and TAG of `<none>` `<none>`. Images like these are called **dangling images**. Since I have created a new version of `quay.io/rhatdan/myimage` with the `podman build `command, the previous image loses that name. You can still use the Podman commands with the image ID, or if the new image doesn’t work, simply use `podman tag` to rename the old image back to `quay.io/rhatdan/myimage`. If the new image works correctly, you can remove the old image with `podman rmi`. These `<none><none>` images tend to build up over time, wasting space, but you can periodically use the `podman image prune` command to remove them.

```sh
podman image prune
```

In [42]:
podman images

REPOSITORY                  TAG         IMAGE ID      CREATED         SIZE
docker.io/lefthand67/test1  latest      b7e73f4c4bfb  13 minutes ago  172 MB
docker.io/library/httpd     latest      59bcd61b45fd  2 weeks ago     172 MB


The `podman build` could really use a chapter or even a book to itself. People build images in thousands of different ways using the commands briefly described here.

- `--tag` is a notable `podman build` option that specifies the image tag or name for the image. Remember that you can always add additional names after you create the image with the `podman tag` command you used in [section 2.2.6](#podman-tag-|-Tagging-images). 
<center>
<b>Table 2.8 Podman image commands</b>
</center>

Command|Man page|Description
-|-|-
build|podman-image-build(1)|Builds an image using instructions from Containerfiles
diff|podman-image-diff(1)|Inspects changes in image’s filesystem
exists|podman-image-exists(1)|Checks whether an image exists
history|podman-image-history(1)|Shows a history of a specified image
import|podman-image-import(1)|Imports a tarball to create a filesystem image
inspect|podman-image-inspect(1)|Displays the configuration of an image
list|podman-image-list(1)|Lists all of the images
load|podman-image-load(1)|Loads image(s) from a tarball
mount|podman-image-mount(1)|Mounts an image’s root filesystem
prune|podman-image-prune(1)|Removes unused images
pull|podman-image-pull(1)|Pulls an image from a registry
push|podman-image-push(1)|Pushes an image to a registry
rm|podman-image-rm(1)|Removes an image
save|podman-image-save(1)|Saves image(s) to an archive
scp|podman-image-scp(1)|Securely copies images to other containers/storage
search|podman-image-search(1)|Searches the registry for an image
sign|podman-image-sign(1)|Signs an image
tag|podman-image-tag(1)|Adds an additional name to a local image
tree|podman-image-tree(1)|Prints the layer hierarchy of an image in a tree format
trust|podman-image-trust(1)|Manages container image trust policy
unmount|podman-image-unmount(1)|Unmounts an image’s root filesystem
untag|podman-image-untag(1)|Removes a name from a local image

# 2.4 Summary

- Podman’s simple command-line interface makes working with containers easy.
- Podman `run`, `stop`, `start`, `ps`, `inspect`, `rm`, and `commit` are all commands for working with containers.
- Podman `pull`, `push`, `login`, and `rmi` are tools for working with images and sharing them via container registries.
- Podman `build` is a great command for automating the build of container images.
- Podman’s command line is based on the Docker CLI and supports it exactly, allowing us to tell people to just alias `Docker = Podman`.
- Podman has additional commands and options to support more advanced concepts like `podman image mount`.

# <b>3. Volumes</b>

This chapter covers

- Using volumes to isolate data from the containerized application
- Sharing content from your host into containers via volumes
- Using volumes with the user namespace and SELinux
- Embedding volumes into container images
- Exploring different types of volumes and the volume commands

Up until now, the containers you have been working with include all their content within the container image. As I described in **chapter 1**, the only thing required to be shared with traditional containers is the Linux kernel. There are several reasons you need to isolate application data from the application, including the following:

- Avoiding embedding actual data for applications such as databases.
- Using the same container image to run multiple environments.
- Reducing overhead and improving storage read/write performance, since volumes write directly to the filesystem, while containers use the **overlay** or **fuse-overlayfs** filesystem to mount their layers. 

> **Overlay** is a layered filesystem, meaning the kernel needs to copy the previous layer entirely to create a new layer, and **fuse-overlayfs** switches each read and write from kernel space to user space and back. All of this creates quite an overhead.

- Sharing content available via network storage.

> NOTE: `bind` mounts remount parts of the file hierarchy in a different location on the filesystem. The files and directories in the `bind mount` are the same as the original (see the `mount` command man page for an explanation of bind mounts). A `bind mount` allows the same content to be accessible in two places, without any additional overhead. It is important to understand that `bind` does not copy the data or create new data.

Supporting volumes also adds complexity, especially concerning security. A lot of the security features of containers prevent the container processes from gaining access to the filesystem outside the container image. In this chapter, you will discover the ways Podman allows you to work around these obstacles.

In [43]:
man mount

MOUNT(8)                     System Administration                    MOUNT(8)

NAME
       mount - mount a filesystem

SYNOPSIS
       mount [-h|-V]

       mount [-l] [-t fstype]

       mount -a [-fFnrsvw] [-t fstype] [-O optlist]

       mount [-fnrsvw] [-o options] device|mountpoint

       mount [-fnrsvw] [-t fstype] [-o options] device mountpoint

       mount --bind|--rbind|--move olddir newdir

       mount
       --make-[shared|slave|private|unbindable|rshared|rslave|rprivate|runbindable]
       mountpoint

DESCRIPTION
       All files accessible in a Unix system are arranged in one big tree, the
       file hierarchy, rooted at /. These files can be spread out over several
       devices. The mount command serves to attach the filesystem found on
       some device to the big file tree. Conversely, the umount(8) command
       will detach it again. The filesystem is used to control how data is
       stored on the device or provided in a virtual way by network or other
     

# 3.1 Using volumes with containers

Let’s go back to your containerized application. Up until now, you have simply embedded the web application data into your container filesystem directly. Recall that in [section 2.1.8](#exec-ing-into-a-container), you used the `podman exec` command to modify the "Hello World" `index.html` data within the container:

In [2]:
podman exec -i myapp bash -c 'cat > /var/www/html/index.html' << _EOF
<html>
 <head>
 </head>
 <body>
  <h1>Hello World</h1>
 </body>
</html>
_EOF

You have made the containerized image more flexible by allowing users to supply their own content for the web service or perhaps to update the web service on the fly (Nope, I had to `podman stop myapp`, `podman rm myapp` and `run` it again, and only then the `index.html` really updated. - VR). At the same time, while this method is possible, it is error prone and not scalable; it is where volumes come in handy.

## `--volume` (`-v`)

Podman allows you to mount host filesystem content into containers using the `podman run` command via the `--volume` (`-v`) option.

The `--volume HOST-DIR:CONTAINER-DIR` option tells Podman to `bind mount` `HOST-DIR` in the host to `CONTAINER-DIR` in the container. Podman supports other kinds of volumes as well, but in this section, I will focus on `bind mount` volumes.

It is possible to mount both files or directories in a single option. Changes of the content on the host will be seen inside the container. Similarly, if container processes change the content inside the container, the changes will be seen on the host.

Let’s look at an example. Create a directory, `html`, in your `home` directory, and then create a new `html/index.html` file in it:

In [44]:
podman ps

CONTAINER ID  IMAGE                              COMMAND           CREATED      STATUS          PORTS                 NAMES
7a818735652c  docker.io/lefthand67/test1:latest  httpd-foreground  6 hours ago  Up 6 hours ago  0.0.0.0:8080->80/tcp  myapp


In [45]:
podman stop myapp

myapp


You have to explicitly remove the image (unlike the `podman build` command), otherwise you will get the `Error: creating container storage: the container name "myapp" is already in use by 7a818735652c8826aea26a5465f2ed64d22dbc36c2841fe21994920df8035eaa. You have to remove that container to be able to reuse that name: that name is already in use`:

In [48]:
podman rm myapp

myapp


In [47]:
mkdir data/html
cat > data/html/index.html << _EOF
<html>
 <head>
 </head>
 <body>
 <h1>No pasaran!</h1>
 </body>
</html>
_EOF

cat data/html/index.html

<html>
 <head>
 </head>
 <body>
 <h1>No pasaran!</h1>
 </body>
</html>


Now launch a container (`user/test1`) with the option `-v ./html:/usr/local/apache2/htdocs`:

```sh
$ podman run -d -v ./data/html:/usr/local/apache2/htdocs:ro,z -p 8080:80 --name myapp docker.io/username/test1
a179372324258aa716367acd9ece8ce1900ab3a28ce19c105e1173b19c406166
```

Notice the extra `:ro,z` fields in the `--volume` option. 
- The `ro` option tells Podman to mount the volume in **read-only** mode. The read-only mount means processes within the container cannot modify any content under `/usr/local/apache2/htdocs`, while processes on the host are still able to modify the content. 

> Podman defaults all volume mounts to **read/write** mode. 

- The `z` option tells Podman to relabel the content to a shared label for use by SELinux (see section 3.1.2).

Now that you have launched the container, open a web browser, and navigate to `localhost:8080` to make sure the changes have taken place.

![Web browser window connecting to the myimage Podman container with volume mounted](../data/images/Screenshot_20240206_020316.png)

Now you can shut down and remove the container you just created. Removing the container does not affect the content at all. The following command removes the latest (`--latest`) container, yours. The `--force` option tells Podman to stop the container and then remove it:

In [49]:
podman rm --latest --force

a179372324258aa716367acd9ece8ce1900ab3a28ce19c105e1173b19c406166


In [50]:
podman images

REPOSITORY                  TAG         IMAGE ID      CREATED      SIZE
docker.io/lefthand67/test1  latest      b7e73f4c4bfb  6 hours ago  172 MB
docker.io/library/httpd     latest      59bcd61b45fd  2 weeks ago  172 MB


Finally, remove the content with this command:

In [51]:
rm -rf data/html

> NOTE: The `--latest` option is not available on Mac and Windows. You must specify the container name or ID. Remote mode is explained in **chapter 9**, and Podman on Mac and Windows is explained in appendixes **E** and **F**.